In [1]:
from ib_insync import *


In [2]:
util.startLoop()

In [3]:
ib = IB()

ib.connect(host='127.0.0.1', port='7497', clientId=13, timeout=60)

<IB connected to 127.0.0.1:7497 clientId=13>

In [4]:
contract2 = ContFuture(
    symbol='N225',  # Symbol for Nikkei 225
    exchange='OSE.JPN',  # Osaka Stock Exchange
    currency='JPY',  # Japanese Yen
    multiplier='1000'  # Typical multiplier for Nikkei 225
)



In [6]:
qualified_contract = ib.qualifyContracts(contract2)
qualified_contract

[ContFuture(conId=652809889, symbol='N225', lastTradeDateOrContractMonth='20250313', multiplier='1000', exchange='OSE.JPN', currency='JPY', localSymbol='160030018', tradingClass='225')]

In [7]:
data = ib.reqMktData(qualified_contract, '', False, False)
data

AttributeError: 'list' object has no attribute 'secType'

In [ ]:
import time
import asyncio
import pandas as pd
from datetime import datetime,time
from ib_broker import *
import credentials
from openpyxl import load_workbook

class IBRKExcel:
    def __init__(self):
        self.path            = credentials.xlsx_path
        self.current_time    = datetime.now()
        # self.excel_data      = pd.read_excel(self.path, sheet_name='Sheet6') # maybe change it to self.path
        # self.excel_url       = f"https://docs.google.com/spreadsheets/d/1gKSi1A1gidZHTm9Vrzm0dEUF8QzcCkCMiSnMMzZtKAA/edit?usp=sharing"
        self.excel_url = "https://docs.google.com/spreadsheets/d/1gKSi1A1gidZHTm9Vrzm0dEUF8QzcCkCMiSnMMzZtKAA/export?format=csv"
        self.excel_data      = pd.read_csv(self.excel_url)
        self.length          = len(self.excel_data)
        self.orderbook       = []
        self.failed_orders   = []
        self.database_path   = credentials.database_path
        # self.square_off_time = datetime.time(11, 45)  # Auto square-off time (11:45)
        self.current_time    = datetime.now().time()
        # self.df = pd.read_excel(self.path, sheet_name="Sheet6")  # Replace with your file path

    def check_excel_changes(self):
        # new_data   = pd.read_excel(self.path, sheet_name='Sheet6')
        new_data = pd.read_csv(self.excel_url)
        new_length = len(new_data)

        if new_length  != self.length:
            self.length     = new_length
            self.excel_data = new_data
            return True
        return False

    def connection_show(self) -> bool:
        host, port = credentials.host, credentials.port
        self.client = IB()
        self.ib = self.client
        connection_print = self.client.connect(host=host, port=port, clientId=13, timeout=60)
        print(connection_print)

    # def format_date_ddmmyyyy(self):
    #     datetime = self.expiry
    #     date,time = datetime.split(" ")
    #     year, day, month = date.split('-')
    #     formatted_date = f"{year}{month.zfill(2)}"
    #     return str(formatted_date)

    def format_date_ddmmyyyy(self,var):
        date,time = var.split(" ")
        year,day,month = date.split('-')
        formatted_date = f"{year}{month.zfill(2)}"
        return str(formatted_date)
    
    def check_for_new_positions(self): # put this in async
        if self.check_excel_changes():
            last_row           = self.excel_data.iloc[-1]
            self.symbol        = 'N225M'
            self.exchange      = 'OSE.JPN' 
            # self.trigger_level = last_row['Trigger_Level_High_Low']
            self.entry_type    = last_row['Entry_Type']
            self.entry_strike  = last_row['Entry_Strike']
            self.strike_type   = last_row['Strike_Type']
            self.expiry        = str(last_row['Expiry'])
            self.target        = last_row['Target']
            self.stop_loss     = last_row['Stop_Loss']
            self.qty           = last_row['Qty']
            self.slicing       = last_row['Slicing']
            self.time_interval = last_row['Time_Interval']
            self.activation    = last_row['Activation']
            if self.strike_type == 'CE':
                self.side = 'SELL'
            else:
                self.side = 'BUY' 

            if self.activation == 1:
                if self.entry_type == "LIMIT":
                    for _ in range(0,int(self.qty/self.slicing),1):
                        datevar = self.expiry
                        # self.contract       = Future(symbol=self.symbol,exchange=self.exchange,lastTradeDateOrContractMonth=self.format_date_ddmmyyyy())
                        datevar_str = datevar.strftime('%Y-%m-%d') if isinstance(datevar, pd.Timestamp) else str(datevar)
                        # Now you can safely split it
                        day,month,year = datevar_str.split('/')  # Ensure the date is in 'YYYY-MM-DD HH:MM:SS' format
                        formatted_date = f"{year}{month.zfill(2)}"
                        self.contract       = Future(symbol=self.symbol,exchange=self.exchange,lastTradeDateOrContractMonth=str(formatted_date))
                        # self.order         = LimitOrder(action=self.side,totalQuantity=str(int(self.slicing)),lmtPrice=str(self.entry_strike)) # maybe change to self.qty/self.slicing
                        self.order         = LimitOrder(action=self.side,totalQuantity=str(int(self.qty)),lmtPrice=str(self.entry_strike)) # maybe change to self.qty/self.slicing
                        self.order.transmit = True
                        self.order_details = self.client.placeOrder(contract=self.contract,order=self.order)
                        print(self.order_details)
                        # print(self.order_details.order.action)
                        # exit(0)
                        print("The order has been placed")
                        time.sleep(self.time_interval)
                else:
                    for _ in range(0,int(self.qty/self.slicing),1):
                        datevar = self.expiry
                        # self.contract       = Future(symbol=self.symbol,exchange=self.exchange,lastTradeDateOrContractMonth=self.format_date_ddmmyyyy())
                        datevar_str = datevar.strftime('%Y-%m-%d') if isinstance(datevar, pd.Timestamp) else str(datevar)
                        # Now you can safely split it
                        day,month,year = datevar_str.split('/')  # Ensure the date is in 'YYYY-MM-DD' format
                        formatted_date = f"{year}{month.zfill(2)}"
                        self.contract       = Future(symbol=self.symbol,exchange=self.exchange,lastTradeDateOrContractMonth=str(formatted_date))
                        # self.order          = MarketOrder(action=self.side,totalQuantity=str(int(self.slicing))) # maybe change to self.qty/self.slicing
                        self.order         = MarketOrder(action=self.side,totalQuantity=str(int(self.qty))) # maybe change to self.qty/self.slicing
                        self.order.transmit = True
                        self.order_details = self.client.placeOrder(contract=self.contract,order=self.order)
                        print(self.order_details)
                        print("The order has been placed")
                        time.sleep(self.time_interval)

    def get_current_market_price_futures(self, contract): 
        """
        Fetches the current market price of the given futures contract.
        """
        self.client.reqMarketDataType(3)
        ticker = self.client.reqMktData(contract, '', False, False)
        self.client.sleep(1)  # Allow data to fetch
        if ticker.last is not None:
            return ticker.last
        elif ticker.close is not None:
            return ticker.close
        return None
    
    def show_details(self):
        result = self.ib.reqOpenOrders()
        return result
    
    def check_for_tp_sl(self, current_price, target_price,stop_loss,action):
        if action == 'PE':
            if current_price >= target_price:  # Target Price hit
                return "SELL"
            elif current_price <= stop_loss:  # Stop Loss hit
                return "SELL"

        elif action == 'CE':
            if current_price >= target_price:  # Target Price hit
                return "BUY"
            elif current_price <= stop_loss:  # Stop Loss hit
                return "BUY"
        return None

    def monitor_tp_sl(self): # put this in async
        self.df = pd.read_excel(self.path, sheet_name="Sheet6")  # Replace with your file path
        
        for i in range(len(self.df)):
            if self.df.loc[i,'Activation'] == 1 and self.df.loc[i,'Strike_Type'] == 'PE':

                datevar = self.df.loc[i, 'Expiry']
                print(datevar)
                # Ensure datevar is a string in 'YYYY-MM-DD' format
                datevar_str = datevar.strftime('%Y-%m-%d') if isinstance(datevar, pd.Timestamp) else str(datevar)
                # Now you can safely split it
                year,day,month = datevar_str.split('-')  # Ensure the date is in 'YYYY-MM-DD HH:MM:SS' format
                formatted_date = f"{year}{month.zfill(2)}"

                # contract      = Future(symbol='N225M',exchange='OSE.JPN',lastTradeDateOrContractMonth='202503') # change this line
                contract      = Future(symbol='N225M',exchange='OSE.JPN',lastTradeDateOrContractMonth=str(formatted_date))
                current_price = self.get_current_market_price_futures(contract)
                if current_price:
                    action = self.check_for_tp_sl(current_price, self.df['Target'].iloc[i],self.df['Stop_Loss'].iloc[i],self.df.loc[i,'Strike_Type'])
                    if action is not None:  
                        order = MarketOrder(action='SELL', totalQuantity=str(self.df['Qty'].iloc[i]))
                        result = self.client.placeOrder(contract, order)
                        print(result)
                        self.df.loc[i, 'Activation'] = 0  
                    else:
                        print("No profit/loss is triggered")

            elif self.df.loc[i,'Activation'] == 1 and self.df.loc[i,'Strike_Type'] == 'CE':

                datevar = self.df.loc[i, 'Expiry']
                print(datevar)
                # Ensure datevar is a string in 'YYYY-MM-DD' format
                datevar_str = datevar.strftime('%Y-%m-%d') if isinstance(datevar, pd.Timestamp) else str(datevar)
                # Now you can safely split it
                year,day,month = datevar_str.split('-')  # Ensure the date is in 'YYYY-MM-DD HH:MM:SS' format
                formatted_date = f"{year}{month.zfill(2)}"

                contract      = Future(symbol='N225M',exchange='OSE.JPN',lastTradeDateOrContractMonth=str(formatted_date)) # change this line
                current_price = self.get_current_market_price_futures(contract)
                if current_price:
                    action = self.check_for_tp_sl(current_price, self.df['Target'].iloc[i],self.df['Stop_Loss'].iloc[i],self.df.loc[i,'Strike_Type'])
                    if action is not None:  
                        order = MarketOrder(action='BUY', totalQuantity=str(self.df['Qty'].iloc[i]))
                        result = self.client.placeOrder(contract, order)
                        print(result)
                        self.df.loc[i, 'Activation'] = 0  
                    else:
                        print("No profit/loss is triggered")

        with pd.ExcelWriter(self.path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
            self.df.to_excel(writer, sheet_name="Sheet6", index=False)
    
    def new_auto_square_off(self): # put this in async
        self.df = pd.read_excel(self.path, sheet_name="Sheet6")
        df = self.df
        current_time = datetime.now().strftime("%H:%M")
        positions = self.client.positions()
        if current_time == "21:04":
            if positions:
                for i in range(len(df)):
                    if self.df.loc[i,'Activation'] == 1:
                        datevar = self.df.loc[i, 'Expiry']
                        print(datevar)
                        # Ensure datevar is a string in 'YYYY-MM-DD' format
                        datevar_str = datevar.strftime('%Y-%m-%d') if isinstance(datevar, pd.Timestamp) else str(datevar)
                        # Now you can safely split it
                        year,day,month = datevar_str.split('-')  # Ensure the date is in 'YYYY-MM-DD HH:MM:SS' format
                        formatted_date = f"{year}{month.zfill(2)}"
                        contract = Future(symbol='N225M', exchange='OSE.JPN', lastTradeDateOrContractMonth=str(formatted_date))
                        if self.df.loc[i, 'Strike_Type'] == 'CE':
                            current_action = 'BUY'
                        else:
                            current_action = 'SELL'
                        order = MarketOrder(action=current_action, totalQuantity=self.df.loc[i, 'Qty'])
                        result = self.client.placeOrder(contract, order)
                        self.df.loc[i, 'Activation'] = 0
            else:
                print("Positions are empty")
        else:
            print("The time is not for closing the market is not yet")
            print(positions)

        with pd.ExcelWriter(self.path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
            self.df.to_excel(writer, sheet_name="Sheet6", index=False)

    def run(self):
        self.connection_show()
        while True:
            self.check_for_new_positions() # working fine 
            # self.new_auto_square_off() # working fine 
            # result = self.show_details() # working fine
            # print(result) # working
            # self.monitor_tp_sl() # working fine
            time.sleep(5)

def main():
    session = IBRKExcel()
    session.run()

if __name__ == "__main__":
    main()

# fix async the rest is good 

# this is fine working code from here in the main code i will just remove the comments to make the code look clean


In [ ]:
def check_for_new_positions(self): # put this in async
        if self.check_excel_changes():
            last_row           = self.excel_data.iloc[-1]
            self.symbol        = 'N225M'
            self.exchange      = 'OSE.JPN' 
            self.trigger_level = last_row['Trigger_Level_High_Low']
            self.entry_type    = last_row['Entry_Type']
            self.entry_strike  = last_row['Entry_Strike']
            self.strike_type   = last_row['Strike_Type']
            self.expiry        = str(last_row['Expiry'])
            self.target        = last_row['Target']
            self.stop_loss     = last_row['Stop_Loss']
            self.qty           = last_row['Qty']
            self.slicing       = last_row['Slicing']
            self.time_interval = last_row['Time_Interval']
            self.activation    = last_row['Activation']
            if self.strike_type == 'CE':
                self.side = 'SELL'
            else:
                self.side = 'BUY' 
            
            datevar = self.expiry
            date,timep = datevar.split(' ')
            year,day,month = date.split('-')
            formatted_date = f"{year}{month.zfill(2)}"
            contract       = Future(symbol=self.symbol,exchange=self.exchange,lastTradeDateOrContractMonth=str(formatted_date))
            if self.activation == 1 and self.trigger_level>self.get_current_market_price_futures(contract) and self.strike_type == "PE":
                if self.entry_type == "LIMIT":
                    for _ in range(0,int(self.qty/self.slicing),1):
                        datevar = self.expiry
                        date,timep = datevar.split(' ')
                        year,day,month = date.split('-')
                        formatted_date = f"{year}{month.zfill(2)}"
                        self.contract       = Future(symbol=self.symbol,exchange=self.exchange,lastTradeDateOrContractMonth=str(formatted_date))
                        # self.order         = LimitOrder(action=self.side,totalQuantity=str(int(self.slicing)),lmtPrice=str(self.entry_strike)) # maybe change to self.qty/self.slicing
                        self.order         = LimitOrder(action=self.side,totalQuantity=str(int(self.qty)),lmtPrice=str(self.entry_strike)) # maybe change to self.qty/self.slicing
                        self.order.transmit = True
                        self.order_details = self.client.placeOrder(contract=self.contract,order=self.order)
                        print(self.order_details)
                        # print(self.order_details.order.action)
                        # exit(0)
                        print("The order has been placed")
                        time.sleep(self.time_interval)
                else:
                    for _ in range(0,int(self.qty/self.slicing),1):
                        datevar = self.expiry
                        date,timep = datevar.split(' ')
                        year,day,month = date.split('-')
                        formatted_date = f"{year}{month.zfill(2)}"
                        print(formatted_date)
                        self.contract       = Future(symbol=self.symbol,exchange=self.exchange,lastTradeDateOrContractMonth=str(formatted_date))
                        # self.order          = MarketOrder(action=self.side,totalQuantity=str(int(self.slicing))) # maybe change to self.qty/self.slicing
                        self.order         = MarketOrder(action=self.side,totalQuantity=str(int(self.qty))) # maybe change to self.qty/self.slicing
                        self.order.transmit = True
                        self.order_details = self.client.placeOrder(contract=self.contract,order=self.order)
                        print(self.order_details)
                        print("The order has been placed")
                        time.sleep(self.time_interval)
            
            elif self.activation == 1 and self.trigger_level<self.get_current_market_price_futures(contract) and self.strike_type == "CE":
                if self.entry_type == "LIMIT":
                    for _ in range(0,int(self.qty/self.slicing),1):
                        datevar = self.expiry
                        date,timep = datevar.split(' ')
                        year,day,month = date.split('-')
                        formatted_date = f"{year}{month.zfill(2)}"
                        self.contract       = Future(symbol=self.symbol,exchange=self.exchange,lastTradeDateOrContractMonth=str(formatted_date))
                        # self.order         = LimitOrder(action=self.side,totalQuantity=str(int(self.slicing)),lmtPrice=str(self.entry_strike)) # maybe change to self.qty/self.slicing
                        self.order         = LimitOrder(action=self.side,totalQuantity=str(int(self.qty)),lmtPrice=str(self.entry_strike)) # maybe change to self.qty/self.slicing
                        self.order.transmit = True
                        self.order_details = self.client.placeOrder(contract=self.contract,order=self.order)
                        print(self.order_details)
                        # print(self.order_details.order.action)
                        # exit(0)
                        print("The order has been placed")
                        time.sleep(self.time_interval)
                else:
                    for _ in range(0,int(self.qty/self.slicing),1):
                        datevar = self.expiry
                        date,timep = datevar.split(' ')
                        year,day,month = date.split('-')
                        formatted_date = f"{year}{month.zfill(2)}"
                        print(formatted_date)
                        self.contract       = Future(symbol=self.symbol,exchange=self.exchange,lastTradeDateOrContractMonth=str(formatted_date))
                        # self.order          = MarketOrder(action=self.side,totalQuantity=str(int(self.slicing))) # maybe change to self.qty/self.slicing
                        self.order         = MarketOrder(action=self.side,totalQuantity=str(int(self.qty))) # maybe change to self.qty/self.slicing
                        self.order.transmit = True
                        self.order_details = self.client.placeOrder(contract=self.contract,order=self.order)
                        print(self.order_details)
                        print("The order has been placed")
                        time.sleep(self.time_interval)
            else:
                print("Some error is here")

# this is code for check_new_posn function

In [ ]:
import time
import asyncio
import pandas as pd
from datetime import datetime
from ib_broker import *
import credentials
from openpyxl import load_workbook
import nest_asyncio

nest_asyncio.apply()

class IBRKExcel:
    def __init__(self):
        self.path            = credentials.xlsx_path
        self.current_time    = datetime.now()
        self.excel_data      = pd.read_excel(self.path, sheet_name='Sheet6') # maybe change it to self.path
        # self.excel_data      = pd.read_csv(self.excel_url)
        self.length          = len(self.excel_data)
        self.orderbook       = []
        self.failed_orders   = []
        self.database_path   = credentials.database_path
        # self.square_off_time = datetime.time(11, 45)  # Auto square-off time (11:45)
        self.current_time    = datetime.now().time()
        # self.df = pd.read_excel(self.path, sheet_name="Sheet6")  # Replace with your file path

    async def check_excel_changes(self):
        new_data   = pd.read_excel(self.path, sheet_name='Sheet6')
        # new_data = pd.read_csv(self.excel_url)
        new_length = len(new_data)

        if new_length  != self.length:
            self.length     = new_length
            self.excel_data = new_data
            return True
        return False

    async def connection_show(self) -> bool:
        host, port = credentials.host, credentials.port
        self.client = IB()
        self.ib = self.client
        # connection_print = self.client.connect(host=host, port=port, clientId=13, timeout=60)
        connection_print = self.client.connect(host=host,port=port,clientId=13,account='DU9727656',timeout=60)
        print(connection_print)

    async def format_date_ddmmyyyy(self,var):
        date,timep = var.split(" ")
        year,day,month = date.split('-')
        formatted_date = f"{year}{month.zfill(2)}"
        return str(formatted_date)
    
    async def check_for_new_positions(self): # put this in async
        if await self.check_excel_changes():
            print("a change on the excel has been made")
            last_row           = self.excel_data.iloc[-1]
            self.symbol        = 'N225M'
            self.exchange      = 'OSE.JPN' 
            self.trigger_level = last_row['Trigger_Level_High_Low']
            self.entry_type    = last_row['Entry_Type']
            self.entry_strike  = last_row['Entry_Strike']
            self.strike_type   = last_row['Strike_Type']
            self.expiry        = str(last_row['Expiry'])
            self.target        = last_row['Target']
            self.stop_loss     = last_row['Stop_Loss']
            self.qty           = last_row['Qty']
            self.slicing       = last_row['Slicing']
            self.time_interval = last_row['Time_Interval']
            self.activation    = last_row['Activation']
            if self.strike_type == 'CE':
                self.side = 'SELL'
            else:
                self.side = 'BUY' 

            if self.activation == 1:

                datevar = self.expiry
                date,timep = datevar.split(' ')
                year,day,month = date.split('-')
                formatted_date = f"{year}{month.zfill(2)}"
                contract = Future(symbol=self.symbol,exchange=self.exchange,lastTradeDateOrContractMonth=str(formatted_date))
                if self.strike_type == "PE" and self.trigger_level< await self.get_current_market_price_futures(contract):
                # if self.strike_type == "PE":
                    if self.entry_type == "LIMIT":
                        for _ in range(0,int(self.qty/self.slicing),1):
                            datevar = self.expiry
                            date,timep = datevar.split(' ')
                            year,day,month = date.split('-')
                            formatted_date = f"{year}{month.zfill(2)}"
                            # print(formatted_date)
                            self.contract       = Future(symbol=self.symbol,exchange=self.exchange,lastTradeDateOrContractMonth=str(formatted_date))
                            self.order         = LimitOrder(action=self.side,totalQuantity=str(int(self.slicing)),lmtPrice=str(self.entry_strike)) # maybe change to self.qty/self.slicing
                            # self.order         = LimitOrder(action=self.side,totalQuantity=str(int(self.qty)),lmtPrice=str(self.entry_strike)) # maybe change to self.qty/self.slicing
                            self.order.account = 'DU9727656'
                            self.order.transmit = True
                            self.order_details = self.client.placeOrder(contract=self.contract,order=self.order)
                            print(self.order_details)
                            print("The order has been placed")
                            # time.sleep(self.time_interval)
                            await asyncio.sleep(self.time_interval)
                    else:
                        for _ in range(0,int(self.qty/self.slicing),1):
                            datevar = self.expiry
                            date,timep = datevar.split(' ')
                            year,day,month = date.split('-')
                            formatted_date = f"{year}{month.zfill(2)}"
                            print(formatted_date)
                            self.contract       = Future(symbol=self.symbol,exchange=self.exchange,lastTradeDateOrContractMonth=str(formatted_date))
                            self.order          = MarketOrder(action=self.side,totalQuantity=str(int(self.slicing))) # maybe change to self.qty/self.slicing
                            # self.order         = MarketOrder(action=self.side,totalQuantity=str(int(self.qty))) # maybe change to self.qty/self.slicing
                            self.order.account = 'DU9727656'
                            self.order.transmit = True
                            self.order_details = self.client.placeOrder(contract=self.contract,order=self.order)
                            print(self.order_details)
                            print("The order has been placed")
                            # time.sleep(self.time_interval)
                            await asyncio.sleep(self.time_interval)

            elif self.strike_type == "CE" and self.trigger_level> await self.get_current_market_price_futures(contract):
                # elif self.strike_type == "CE":
                    if self.entry_type == "LIMIT":
                        for _ in range(0,int(self.qty/self.slicing),1):
                            datevar = self.expiry
                            date,timep = datevar.split(' ')
                            year,day,month = date.split('-')
                            formatted_date = f"{year}{month.zfill(2)}"
                            self.contract       = Future(symbol=self.symbol,exchange=self.exchange,lastTradeDateOrContractMonth=str(formatted_date))
                            self.order         = LimitOrder(action=self.side,totalQuantity=str(int(self.slicing)),lmtPrice=str(self.entry_strike)) # maybe change to self.qty/self.slicing
                            # self.order         = LimitOrder(action=self.side,totalQuantity=str(int(self.qty)),lmtPrice=str(self.entry_strike)) # maybe change to self.qty/self.slicing
                            self.order.account = 'DU9727656'
                            self.order.transmit = True
                            self.order_details = self.client.placeOrder(contract=self.contract,order=self.order)
                            print(self.order_details)
                            print("The order has been placed")
                            # time.sleep(self.time_interval)
                            await asyncio.sleep(self.time_interval)
                    else:
                        for _ in range(0,int(self.qty/self.slicing),1):
                            datevar = self.expiry
                            date,timep = datevar.split(' ')
                            year,day,month = date.split('-')
                            formatted_date = f"{year}{month.zfill(2)}"
                            print(formatted_date)
                            self.contract       = Future(symbol=self.symbol,exchange=self.exchange,lastTradeDateOrContractMonth=str(formatted_date))
                            self.order          = MarketOrder(action=self.side,totalQuantity=str(int(self.slicing))) # maybe change to self.qty/self.slicing
                            # self.order         = MarketOrder(action=self.side,totalQuantity=str(int(self.qty))) # maybe change to self.qty/self.slicing
                            self.order.account = 'DU9727656'
                            self.order.transmit = True
                            self.order_details = self.client.placeOrder(contract=self.contract,order=self.order)
                            print(self.order_details)
                            print("The order has been placed")
                            # time.sleep(self.time_interval)
                            await asyncio.sleep(self.time_interval)
            else:
                    print("The trigger price has not being triggered")
        else:
            print("No changes in excel")

    async def get_current_market_price_futures(self, contract): 
        """
        Fetches the current market price of the given futures contract.
        """
        self.client.reqMarketDataType(3)
        ticker = self.client.reqMktData(contract, '', False, False)
        self.client.sleep(1)  # Allow data to fetch
        if ticker.last is not None: # if not working then use ticker.close
            return ticker.last
        if ticker.close is not None: # if not working then use ticker.close
            return ticker.close
        
        print(ticker.last)
        return None
    
    async def show_details(self):
        result = self.ib.reqOpenOrders()
        return result
    
    async def check_for_tp_sl(self, current_price, target_price,stop_loss,action): # function is corrected
        if action == 'PE':
            if current_price >= target_price:  # Target Price hit
                return "SELL"
            elif current_price <= stop_loss:  # Stop Loss hit
                return "SELL"

        elif action == 'CE':
            if current_price <= target_price:  # corrected
                return "BUY"
            elif current_price >= stop_loss:  # corrected
                return "BUY"
        return None

    async def monitor_tp_sl(self): # put this in async
        self.df = pd.read_excel(self.path, sheet_name="Sheet6")  
        for i in range(len(self.df)):
            if self.df.loc[i,'Activation'] == 1 and self.df.loc[i,'Strike_Type'] == 'PE':
                datevar = self.df.loc[i, 'Expiry']
                datevar_str = datevar.strftime('%Y-%m-%d') if isinstance(datevar, pd.Timestamp) else str(datevar)
                year,day,month = datevar_str.split('-')  # Ensure the date is in 'YYYY-MM-DD HH:MM:SS' format
                formatted_date = f"{year}{month.zfill(2)}"
                contract      = Future(symbol='N225M',exchange='OSE.JPN',lastTradeDateOrContractMonth=str(formatted_date))
                current_price = await self.get_current_market_price_futures(contract)
                if current_price:
                    action = await self.check_for_tp_sl(current_price, self.df['Target'].iloc[i],self.df['Stop_Loss'].iloc[i],self.df.loc[i,'Strike_Type'])
                    if action is not None:  
                        order = MarketOrder(action='SELL', totalQuantity=str(self.df['Qty'].iloc[i]))
                        order.account = 'DU9727656'
                        order.transmit = True
                        result = self.client.placeOrder(contract, order)
                        print(result)
                        self.df.loc[i, 'Activation'] = 0
                        print("One position is being closed")  
                    else:
                        print("No profit/loss is triggered")

            elif self.df.loc[i,'Activation'] == 1 and self.df.loc[i,'Strike_Type'] == 'CE':

                datevar = self.df.loc[i, 'Expiry']
                datevar_str = datevar.strftime('%Y-%m-%d') if isinstance(datevar, pd.Timestamp) else str(datevar)
                year,day,month = datevar_str.split('-')  # Ensure the date is in 'YYYY-MM-DD HH:MM:SS' format
                formatted_date = f"{year}{month.zfill(2)}"

                contract      = Future(symbol='N225M',exchange='OSE.JPN',lastTradeDateOrContractMonth=str(formatted_date)) # change this line
                current_price = await self.get_current_market_price_futures(contract)
                if current_price:
                    action = await self.check_for_tp_sl(current_price, self.df['Target'].iloc[i],self.df['Stop_Loss'].iloc[i],self.df.loc[i,'Strike_Type'])
                    if action is not None:  
                        order = MarketOrder(action='BUY', totalQuantity=str(self.df['Qty'].iloc[i]))
                        order.account = 'DU9727656'
                        order.transmit = True
                        result = self.client.placeOrder(contract, order)
                        print(result)
                        self.df.loc[i, 'Activation'] = 0  
                    else:
                        print("No profit/loss is triggered")

        with pd.ExcelWriter(self.path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
            self.df.to_excel(writer, sheet_name="Sheet6", index=False)
    
    async def new_auto_square_off(self): # put this in async
        self.df = pd.read_excel(self.path, sheet_name="Sheet6")
        df = self.df
        current_time = datetime.now().strftime("%H:%M")
        positions = self.client.positions()
        if current_time == "13:45":
            if positions:
                for i in range(len(df)):
                    if self.df.loc[i,'Activation'] == 1:
                        datevar = self.df.loc[i, 'Expiry']
                        print(datevar)
                        # Ensure datevar is a string in 'YYYY-MM-DD' format
                        datevar_str = datevar.strftime('%Y-%m-%d') if isinstance(datevar, pd.Timestamp) else str(datevar)
                        # Now you can safely split it
                        year,day,month = datevar_str.split('-')  # Ensure the date is in 'YYYY-MM-DD HH:MM:SS' format
                        formatted_date = f"{year}{month.zfill(2)}"
                        contract = Future(symbol='N225M', exchange='OSE.JPN', lastTradeDateOrContractMonth=str(formatted_date))
                        if self.df.loc[i, 'Strike_Type'] == 'CE':
                            current_action = 'BUY'
                        else:
                            current_action = 'SELL'
                        order = MarketOrder(action=current_action, totalQuantity=self.df.loc[i, 'Qty'])
                        order.account = 'DU9727656'
                        order.transmit = True
                        result = self.client.placeOrder(contract, order)
                        self.df.loc[i, 'Activation'] = 0
            else:
                print("Positions are empty")
        else:
            print("The time is not for closing the market is not yet")

        with pd.ExcelWriter(self.path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
            self.df.to_excel(writer, sheet_name="Sheet6", index=False)

    async def run(self):
        print("The process has started")
        await self.connection_show()
        while True:
            await asyncio.gather(self.check_for_new_positions(),self.new_auto_square_off(),self.monitor_tp_sl())
            await asyncio.sleep(10)

if __name__ == "__main__":
    session = IBRKExcel()
    asyncio.run(session.run())


In [ ]:
if count<3:
    self.order_details = self.client.placeOrder(contract=self.contract,order=self.order)
    await asyncio.sleep(3)
    if self.order_details.orderStatus.status != "Filled":
        self.client.cancelOrder(order=self.order_details)
        self.order_details = self.client.placeOrder(contract = self.contract,order=self.order)
        await asyncio.sleep(3)
        count = count+1
    else:
        contract = MarketOrder()
        self.client.placeOrder(contract = self.contract,order=self.order)
        count = 0

In [ ]:
import time
import asyncio
import pandas as pd
from datetime import datetime,time
from ib_broker import *
import credentials
from openpyxl import load_workbook

class IBRKExcel:
    def __init__(self):
        self.path            = credentials.xlsx_path
        self.current_time    = datetime.now()
        self.excel_url = "https://docs.google.com/spreadsheets/d/1gKSi1A1gidZHTm9Vrzm0dEUF8QzcCkCMiSnMMzZtKAA/export?format=csv"
        self.excel_data      = pd.read_csv(self.excel_url)
        self.length          = len(self.excel_data)
        self.orderbook       = []
        self.failed_orders   = []
        self.database_path   = credentials.database_path
        self.current_time    = datetime.now().time()

    def check_excel_changes(self):
        new_data = pd.read_csv(self.excel_url)
        new_length = len(new_data)

        if new_length  != self.length:
            self.length     = new_length
            self.excel_data = new_data
            return True
        return False

    def connection_show(self) -> bool:
        host, port = credentials.host, credentials.port
        self.client = IB()
        self.ib = self.client
        connection_print = self.client.connect(host=host, port=port, clientId=13, timeout=60)
        print(connection_print)

    def format_date_ddmmyyyy(self,var):
        date,time = var.split(" ")
        year,day,month = date.split('-')
        formatted_date = f"{year}{month.zfill(2)}"
        return str(formatted_date)
    
    def check_for_new_positions(self): # put this in async
        if self.check_excel_changes():
            last_row           = self.excel_data.iloc[-1]
            self.symbol        = 'N225M'
            self.exchange      = 'OSE.JPN' 
            self.trigger_level = last_row['Trigger_Level_High_Low']
            self.entry_type    = last_row['Entry_Type']
            self.entry_strike  = last_row['Entry_Strike']
            self.strike_type   = last_row['Strike_Type']
            self.expiry        = str(last_row['Expiry'])
            self.target        = last_row['Target']
            self.stop_loss     = last_row['Stop_Loss']
            self.qty           = last_row['Qty']
            self.slicing       = last_row['Slicing']
            self.time_interval = last_row['Time_Interval']
            self.activation    = last_row['Activation']
            if self.strike_type == 'CE':
                self.side = 'SELL'
            else:
                self.side = 'BUY' 
            self.i = 0
            if self.activation == 1:
                if self.entry_type == "LIMIT":
                    # for _ in range(0,int(self.qty/self.slicing),1):
                    for i in range(0,int(self.qty/self.slicing)):
                        datevar = self.expiry
                        datevar_str = datevar.strftime('%Y-%m-%d') if isinstance(datevar, pd.Timestamp) else str(datevar)
                        day,month,year = datevar_str.split('/')  
                        formatted_date = f"{year}{month.zfill(2)}"
                        self.contract       = Future(symbol=self.symbol,exchange=self.exchange,lastTradeDateOrContractMonth=str(formatted_date))
                        # self.order         = LimitOrder(action=self.side,totalQuantity=str(int(self.slicing)),lmtPrice=str(self.entry_strike)) # maybe change to self.qty/self.slicing
                        self.order         = LimitOrder(action=self.side,totalQuantity=str(int(self.qty)),lmtPrice=str(self.entry_strike)) 
                        self.order.transmit = True
                        self.order_details = self.client.placeOrder(contract=self.contract,order=self.order)
                        print(self.order_details)
                        print("The order has been placed")
                        time.sleep(self.time_interval*2)
                        
                else:
                    # for _ in range(0,int(self.qty/self.slicing),1):
                    for i in range(0,int(self.qty/self.slicing)):
                        datevar = self.expiry
                        datevar_str = datevar.strftime('%Y-%m-%d') if isinstance(datevar, pd.Timestamp) else str(datevar)
                        day,month,year = datevar_str.split('/')  
                        formatted_date = f"{year}{month.zfill(2)}"
                        self.contract       = Future(symbol=self.symbol,exchange=self.exchange,lastTradeDateOrContractMonth=str(formatted_date))
                        # self.order         = MarketOrder(action=self.side,totalQuantity=str(int(self.slicing))) # maybe change to self.qty/self.slicing
                        self.order         = MarketOrder(action=self.side,totalQuantity=str(int(self.qty))) 
                        self.order.transmit = True
                        self.order_details = self.client.placeOrder(contract=self.contract,order=self.order)
                        print(self.order_details)
                        print("The order has been placed")
                        time.sleep(self.time_interval*2)


    def get_current_market_price_futures(self, contract): 
        """
        Fetches the current market price of the given futures contract.
        """
        self.client.reqMarketDataType(3)
        ticker = self.client.reqMktData(contract, '', False, False)
        self.client.sleep(1)  
        if ticker.last is not None:
            return ticker.last
        elif ticker.close is not None:
            return ticker.close
        return None
    
    def show_details(self):
        result = self.ib.reqOpenOrders()
        return result
    
    def check_for_tp_sl(self, current_price, target_price,stop_loss,action):
        if action == 'PE':
            if current_price >= target_price:  # Target Price hit
                return "SELL"
            elif current_price <= stop_loss:  # Stop Loss hit
                return "SELL"

        elif action == 'CE':
            if current_price >= target_price:  # Target Price hit
                return "BUY"
            elif current_price <= stop_loss:  # Stop Loss hit
                return "BUY"
        return None

    def monitor_tp_sl(self): # put this in async
        self.df = pd.read_csv(self.excel_url)
        
        for i in range(len(self.df)):
            if self.df.loc[i,'Activation'] == 1 and self.df.loc[i,'Strike_Type'] == 'PE':

                datevar = self.df.loc[i, 'Expiry']
                datevar_str = datevar.strftime('%Y-%m-%d') if isinstance(datevar, pd.Timestamp) else str(datevar)
                year,day,month = datevar_str.split('/')  
                formatted_date = f"{year}{month.zfill(2)}"
                contract      = Future(symbol='N225M',exchange='OSE.JPN',lastTradeDateOrContractMonth=str(formatted_date))
                current_price = self.get_current_market_price_futures(contract)
                if current_price:
                    action = self.check_for_tp_sl(current_price, self.df['Target'].iloc[i],self.df['Stop_Loss'].iloc[i],self.df.loc[i,'Strike_Type'])
                    if action is not None:  
                        order = MarketOrder(action='SELL', totalQuantity=str(self.df['Qty'].iloc[i]))
                        result = self.client.placeOrder(contract, order)
                        print(result)
                        self.df.loc[i, 'Activation'] = 0  
                    else:
                        print("No profit/loss is triggered")

            elif self.df.loc[i,'Activation'] == 1 and self.df.loc[i,'Strike_Type'] == 'CE':

                datevar = self.df.loc[i, 'Expiry']
                print(datevar)
                datevar_str = datevar.strftime('%Y-%m-%d') if isinstance(datevar, pd.Timestamp) else str(datevar)
                year,day,month = datevar_str.split('/')  
                formatted_date = f"{year}{month.zfill(2)}"

                contract      = Future(symbol='N225M',exchange='OSE.JPN',lastTradeDateOrContractMonth=str(formatted_date)) # change this line
                current_price = self.get_current_market_price_futures(contract)
                if current_price:
                    action = self.check_for_tp_sl(current_price, self.df['Target'].iloc[i],self.df['Stop_Loss'].iloc[i],self.df.loc[i,'Strike_Type'])
                    if action is not None:  
                        order = MarketOrder(action='BUY', totalQuantity=str(self.df['Qty'].iloc[i]))
                        result = self.client.placeOrder(contract, order)
                        print(result)
                        self.df.loc[i, 'Activation'] = 0  
                    else:
                        print("No profit/loss is triggered")

        with pd.ExcelWriter(self.path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
            self.df.to_excel(writer, sheet_name="Sheet6", index=False)
    
    def new_auto_square_off(self): # put this in async
        self.df = pd.read_csv(self.excel_url)
        df = self.df
        current_time = datetime.now().strftime("%H:%M")
        positions = self.client.positions()
        if current_time == "10:20":
            if positions:
                for i in range(len(df)):
                    if self.df.loc[i,'Activation'] == 1:
                        datevar = self.df.loc[i, 'Expiry']
                        print(datevar)
                        datevar_str = datevar.strftime('%Y-%m-%d') if isinstance(datevar, pd.Timestamp) else str(datevar)
                        day,month,year = datevar_str.split('/')  # Ensure the date is in 'YYYY-MM-DD HH:MM:SS' format
                        formatted_date = f"{year}{month.zfill(2)}"
                        contract = Future(symbol='N225M', exchange='OSE.JPN', lastTradeDateOrContractMonth=str(formatted_date))
                        if self.df.loc[i, 'Strike_Type'] == 'CE':
                            current_action = 'BUY'
                        else:
                            current_action = 'SELL'
                        order = MarketOrder(action=current_action, totalQuantity=self.df.loc[i, 'Qty'])
                        result = self.client.placeOrder(contract, order)
                        self.df.loc[i, 'Activation'] = 0
            else:
                print("Positions are empty")
        else:
            print("The time is not for closing the market is not yet")
            print(positions)

        with pd.ExcelWriter(self.path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
            self.df.to_excel(writer, sheet_name="Sheet6", index=False)

    def run(self):
        self.connection_show()
        while True:
            self.check_for_new_positions() # working fine 
            # self.new_auto_square_off() # working fine 
            # result = self.show_details() # working fine
            # print(result) # working
            # self.monitor_tp_sl() # working fine
            time.sleep(5)

def main():
    session = IBRKExcel()
    session.run()

if __name__ == "__main__":
    main()

# fix async the rest is good 



In [ ]:
import time
import asyncio
import pandas as pd
from datetime import datetime
from ib_broker import *
import credentials
from openpyxl import load_workbook
import nest_asyncio

nest_asyncio.apply()

class IBRKExcel:
    def __init__(self):
        self.path            = credentials.xlsx_path
        self.current_time    = datetime.now()
        self.excel_data      = pd.read_excel(self.path, sheet_name='Sheet6') 
        self.length          = len(self.excel_data)
        self.orderbook       = []
        self.failed_orders   = []
        self.database_path   = credentials.database_path
        # self.square_off_time = datetime.time(11, 45)  # Auto square-off time (11:45)
        self.current_time    = datetime.now().time()
        # self.df = pd.read_excel(self.path, sheet_name="Sheet6")  # Replace with your file path

    async def check_excel_changes(self):
        new_data   = pd.read_excel(self.path, sheet_name='Sheet6')
        new_length = len(new_data)

        if new_length  != self.length:
            self.length     = new_length
            self.excel_data = new_data
            return True
        return False

    async def connection_show(self) -> bool:
        host, port = credentials.host, credentials.port
        self.client = IB()
        self.ib = self.client
        connection_print = self.client.connect(host=host,port=port,clientId=13,account='DU9727656',timeout=60)
        print(connection_print)

    async def format_date_ddmmyyyy(self,var):
        date,timep = var.split(" ")
        year,day,month = date.split('-')
        formatted_date = f"{year}{month.zfill(2)}"
        return str(formatted_date)

    async def check_for_new_positions(self): # put this in async
        if await self.check_excel_changes():
            print("a change on the excel has been made")
            length   = len(pd.read_excel(self.path, sheet_name='Sheet6'))
            for i in range(length):
                if self.excel_data.loc[i,'Activation'] == 1:
                    row           = self.excel_data.iloc[i]
                    self.symbol        = 'N225M'
                    self.exchange      = 'OSE.JPN' 
                    self.trigger_level = row['Trigger_Level_High_Low']
                    self.entry_type    = row['Entry_Type']
                    self.entry_strike  = row['Entry_Strike']
                    self.strike_type   = row['Strike_Type']
                    self.expiry        = str(row['Expiry'])
                    self.target        = row['Target']
                    self.stop_loss     = row['Stop_Loss']
                    self.qty           = row['Qty']
                    self.slicing       = row['Slicing']
                    self.time_interval = row['Time_Interval']
                    self.activation    = row['Activation']
                    if self.strike_type == 'CE':
                        self.side = 'SELL'
                    else:
                        self.side = 'BUY' 

                    datevar = self.expiry
                    date,timep = datevar.split(' ')
                    year,day,month = date.split('-')
                    formatted_date = f"{year}{month.zfill(2)}"
                    contract = Future(symbol=self.symbol,exchange=self.exchange,lastTradeDateOrContractMonth=str(formatted_date))
                    if self.strike_type == "PE" and self.trigger_level< await self.get_current_market_price_futures(contract):
                        if self.entry_type == "LIMIT":
                            for _ in range(0,int(self.qty/self.slicing),1):
                                datevar = self.expiry
                                date,timep = datevar.split(' ')
                                year,day,month = date.split('-')
                                formatted_date = f"{year}{month.zfill(2)}"
                                self.contract       = Future(symbol=self.symbol,exchange=self.exchange,lastTradeDateOrContractMonth=str(formatted_date))
                                self.order         = LimitOrder(action=self.side,totalQuantity=str(int(self.slicing)),lmtPrice=str(self.entry_strike)) 
                                self.order.account = 'DU9727656'
                                self.order.transmit = True
                                print("The first placed order ")
                                self.order_details = self.client.placeOrder(contract=self.contract,order=self.order)
                                await asyncio.sleep(3)
                                print(self.order_details)
                                if self.order_details.orderStatus.status != "Filled":
                                    print("The order failed for the first time")
                                    fail_status = self.client.cancelOrder(order=self.order_details.orderStatus)
                                    print(fail_status)
                                    self.order=MarketOrder(action=self.side,totalQuantity=str(int(self.slicing)))
                                    self.final_order_details = self.client.placeOrder(contract=self.contract,order=self.order)
                                    print(self.final_order_details)
                                    print("The final order has been placed")

                                print("The order has been placed")
                            self.excel_data.loc[i,'Activation'] = -1 
                            with pd.ExcelWriter(self.path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
                                self.excel_data.to_excel(writer, sheet_name="Sheet6", index=False)
                            await asyncio.sleep(self.time_interval)

                        else:
                            for _ in range(0,int(self.qty/self.slicing),1):
                                datevar = self.expiry
                                date,timep = datevar.split(' ')
                                year,day,month = date.split('-')
                                formatted_date = f"{year}{month.zfill(2)}"
                                print(formatted_date)
                                self.contract       = Future(symbol=self.symbol,exchange=self.exchange,lastTradeDateOrContractMonth=str(formatted_date))
                                self.order          = MarketOrder(action=self.side,totalQuantity=str(int(self.slicing))) 
                                self.order.account = 'DU9727656'
                                self.order.transmit = True
                                self.order_details = self.client.placeOrder(contract=self.contract,order=self.order)
                                await asyncio.sleep(3)
                                print("The order has been placed")
                                self.excel_data.loc[i,'Activation'] = -1 
                                with pd.ExcelWriter(self.path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
                                    self.excel_data.to_excel(writer, sheet_name="Sheet6", index=False)
                                await asyncio.sleep(self.time_interval)

                    elif self.strike_type == "CE" and self.trigger_level> await self.get_current_market_price_futures(contract):
                        if self.entry_type == "LIMIT":
                            for _ in range(0,int(self.qty/self.slicing),1):
                                datevar = self.expiry
                                date,timep = datevar.split(' ')
                                year,day,month = date.split('-')
                                formatted_date = f"{year}{month.zfill(2)}"
                                self.contract       = Future(symbol=self.symbol,exchange=self.exchange,lastTradeDateOrContractMonth=str(formatted_date))
                                self.order         = LimitOrder(action=self.side,totalQuantity=str(int(self.slicing)),lmtPrice=str(self.entry_strike)) 
                                self.order.account = 'DU9727656'
                                self.order.transmit = True
                                self.order_details = self.client.placeOrder(contract=self.contract,order=self.order)
                                await asyncio.sleep(3)
                                
                                if self.order_details.orderStatus.status != "Filled":
                                    print("The order failed for the first time")
                                    # fail_status = self.client.cancelOrder(order=self.order_details.orderStatus,manualCancelOrderTime='2s')
                                    fail_status = self.client.cancelOrder(order=self.order_details.orderStatus)
                                    print(fail_status)
                                    self.order_details = self.client.placeOrder(contract=self.contract,order=self.order)
                                    await asyncio.sleep(3)
                                    if self.order_details.orderStatus.status != "Filled":
                                        fail_status = self.client.cancelOrder(order=self.order_details.orderStatus)
                                        print(fail_status)
                                        self.order=MarketOrder(action=self.side,totalQuantity=str(int(self.slicing)))
                                        self.order_details = self.client.placeOrder(contract=self.contract,order=self.order)
                                        print("finally a market order has been placed")
                                        print(self.order_details)
                                        print("The order has been placed")

                            self.excel_data.loc[i,'Activation'] = -1 # now use excelwriter fn
                            with pd.ExcelWriter(self.path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
                                self.excel_data.to_excel(writer, sheet_name="Sheet6", index=False)
                            await asyncio.sleep(self.time_interval)

                        else:
                            for _ in range(0,int(self.qty/self.slicing),1):
                                datevar = self.expiry
                                date,timep = datevar.split(' ')
                                year,day,month = date.split('-')
                                formatted_date = f"{year}{month.zfill(2)}"
                                print(formatted_date)
                                self.contract       = Future(symbol=self.symbol,exchange=self.exchange,lastTradeDateOrContractMonth=str(formatted_date))
                                self.order          = MarketOrder(action=self.side,totalQuantity=str(int(self.slicing))) 
                                self.order.account = 'DU9727656'
                                self.order.transmit = True
                                self.order_details = self.client.placeOrder(contract=self.contract,order=self.order)
                                await asyncio.sleep(3)
                                print(self.order_details)
                                print("The order has been placed")
                                self.excel_data.loc[i,'Activation'] = -1 # now use excelwriter fn
                                with pd.ExcelWriter(self.path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
                                    self.excel_data.to_excel(writer, sheet_name="Sheet6", index=False)
                                await asyncio.sleep(self.time_interval)
                    else:
                        print("The trigger price has not being triggered")
        else:
            print("No changes in excel")

    async def get_current_market_price_futures(self, contract): 
        """
        Fetches the current market price of the given futures contract.
        """
        self.client.reqMarketDataType(3)
        ticker = self.client.reqMktData(contract, '', False, False)
        self.client.sleep(1)  
        if ticker.last is not None: 
            return ticker.last
        if ticker.close is not None: 
            return ticker.close
        
        print(ticker.last)
        return None
    
    async def show_details(self):
        result = self.ib.reqOpenOrders()
        return result
    
    async def check_for_tp_sl(self, current_price, target_price,stop_loss,action): 
        if action == 'PE':
            if current_price >= target_price:  # Target Price hit
                return "SELL"
            elif current_price <= stop_loss:  # Stop Loss hit
                return "SELL"

        elif action == 'CE':
            if current_price <= target_price:  # corrected
                return "BUY"
            elif current_price >= stop_loss:  # corrected
                return "BUY"
        return None

    async def monitor_tp_sl(self): 
        self.df = pd.read_excel(self.path, sheet_name="Sheet6")  
        for i in range(len(self.df)):
            if self.df.loc[i,'Activation'] == -1 and self.df.loc[i,'Strike_Type'] == 'PE':
                datevar = self.df.loc[i, 'Expiry']
                # Ensure datevar is a string in 'YYYY-MM-DD' format
                datevar_str = datevar.strftime('%Y-%m-%d') if isinstance(datevar, pd.Timestamp) else str(datevar)
                year,day,month = datevar_str.split('-')  # Ensure the date is in 'YYYY-MM-DD HH:MM:SS' format
                formatted_date = f"{year}{month.zfill(2)}"
                contract      = Future(symbol='N225M',exchange='OSE.JPN',lastTradeDateOrContractMonth=str(formatted_date))
                current_price = await self.get_current_market_price_futures(contract)
                if current_price:
                    action = await self.check_for_tp_sl(current_price, self.df['Target'].iloc[i],self.df['Stop_Loss'].iloc[i],self.df.loc[i,'Strike_Type'])
                    if action is not None:  
                        order = MarketOrder(action='SELL', totalQuantity=str(self.df['Qty'].iloc[i]))
                        order.account = 'DU9727656'
                        order.transmit = True
                        result = self.client.placeOrder(contract, order)
                        print(result)
                        self.df.loc[i, 'Activation'] = 0
                        print("One position is being closed")  
                    else:
                        print("No profit/loss is triggered")

            elif self.df.loc[i,'Activation'] == -1 and self.df.loc[i,'Strike_Type'] == 'CE':

                datevar = self.df.loc[i, 'Expiry']
                print(datevar)
                # Ensure datevar is a string in 'YYYY-MM-DD' format
                datevar_str = datevar.strftime('%Y-%m-%d') if isinstance(datevar, pd.Timestamp) else str(datevar)
                year,day,month = datevar_str.split('-')  # Ensure the date is in 'YYYY-MM-DD HH:MM:SS' format
                formatted_date = f"{year}{month.zfill(2)}"

                contract      = Future(symbol='N225M',exchange='OSE.JPN',lastTradeDateOrContractMonth=str(formatted_date)) # change this line
                current_price = await self.get_current_market_price_futures(contract)
                if current_price:
                    action = await self.check_for_tp_sl(current_price, self.df['Target'].iloc[i],self.df['Stop_Loss'].iloc[i],self.df.loc[i,'Strike_Type'])
                    if action is not None:  
                        order = MarketOrder(action='BUY', totalQuantity=str(self.df['Qty'].iloc[i]))
                        order.account = 'DU9727656'
                        order.transmit = True
                        result = self.client.placeOrder(contract, order)
                        print(result)
                        self.df.loc[i, 'Activation'] = 0  
                    else:
                        print("No profit/loss is triggered")

        with pd.ExcelWriter(self.path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
            self.df.to_excel(writer, sheet_name="Sheet6", index=False)
    
    async def new_auto_square_off(self): # put this in async
        self.df = pd.read_excel(self.path, sheet_name="Sheet6")
        df = self.df
        current_time = datetime.now().strftime("%H:%M")
        positions = self.client.positions()
        if current_time > "9:10":
            if positions:
                for i in range(len(df)):
                    if self.df.loc[i,'Activation'] == -1:
                        datevar = self.df.loc[i, 'Expiry']
                        # Ensure datevar is a string in 'YYYY-MM-DD' format
                        datevar_str = datevar.strftime('%Y-%m-%d') if isinstance(datevar, pd.Timestamp) else str(datevar)
                        year,day,month = datevar_str.split('-')  # Ensure the date is in 'YYYY-MM-DD HH:MM:SS' format
                        formatted_date = f"{year}{month.zfill(2)}"
                        contract = Future(symbol='N225M', exchange='OSE.JPN', lastTradeDateOrContractMonth=str(formatted_date))
                        if self.df.loc[i, 'Strike_Type'] == 'CE':
                            current_action = 'BUY'
                        else:
                            current_action = 'SELL'
                        order = MarketOrder(action=current_action, totalQuantity=self.df.loc[i, 'Qty'])
                        order.account = 'DU9727656'
                        order.transmit = True
                        result = self.client.placeOrder(contract, order)
                        self.df.loc[i, 'Activation'] = 0
            else:
                print("Positions are empty")
        else:
            print("The time is not for closing the market is not yet")

        with pd.ExcelWriter(self.path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
            self.df.to_excel(writer, sheet_name="Sheet6", index=False)

    async def run(self):
        print("The process has started")
        await self.connection_show()
        while True:
            await asyncio.gather(self.check_for_new_positions(),self.new_auto_square_off(),self.monitor_tp_sl())
            await asyncio.sleep(10)

if __name__ == "__main__":
    session = IBRKExcel()
    asyncio.run(session.run())



In [4]:
import pandas as pd

google_sheet_id = "16bpyg4FMgbd22_9FwOCrCqmDqg1Y7Ulff38w-xX1kXM"
sheet_name = "Sheet1"  # Change this to your actual sheet name

# Correct export link for CSV format
csv_url = f"https://docs.google.com/spreadsheets/d/{google_sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"

# Read the CSV file
df = pd.read_csv(csv_url)

# Print the dataframe
print(df)


     Trigger_Level_High_Low Entry_Type  Entry_Strike Strike_Type    Expiry  \
0                     38700     MARKET         38750          PE  ########   
1                     38700     MARKET         38750          PE  ########   
2                     38700     MARKET         38750          PE  ########   
3                     38700     MARKET         38750          PE  ########   
4                     38700     MARKET         38750          PE  ########   
..                      ...        ...           ...         ...       ...   
320                   36800     MARKET         36830          CE  ########   
321                   36800     MARKET         36830          CE  ########   
322                   36800     MARKET         36830          CE  ########   
323                   36850      LIMIT         36860          CE  ########   
324                   36850      LIMIT         36860          CE  ########   

     Target  Stop_Loss  Qty  Slicing  Time_Interval  Activation

In [ ]:
import time
import asyncio
import pandas as pd
from datetime import datetime
from ib_broker import *
import credentials
from openpyxl import load_workbook
import nest_asyncio

nest_asyncio.apply()

class IBRKExcel:
    def __init__(self):
        self.symbol          = 'N225M'
        self.exchange        = 'OSE.JPN'
        self.path            = credentials.xlsx_path
        self.current_time    = datetime.now()
        self.excel_data      = pd.read_excel(self.path, sheet_name='Sheet1') 
        self.length          = len(self.excel_data)
        self.orderbook       = []
        self.failed_orders   = []
        # self.database_path   = credentials.database_path
        self.upper_trigger   = 100000000
        self.lower_trigger   = -100000000
        self.current_time    = datetime.now().time()

    async def check_excel_changes(self):
        new_data   = pd.read_excel(self.path, sheet_name='Sheet1')
        new_length = len(new_data)

        if new_length  != self.length:
            self.length     = new_length
            self.excel_data = new_data
            return True
        return False

    async def connection_show(self) -> bool:
        host, port = credentials.host, credentials.port
        self.client = IB()
        self.ib = self.client
        connection_print = self.client.connect(host=host,port=port,clientId=13,account='DU9727656',timeout=60)
        print(connection_print)

    async def check_for_new_positions(self): # put this in async
        if await self.check_excel_changes():
            print("a change on the excel has been made")
            length   = len(pd.read_excel(self.path, sheet_name='Sheet1'))
            for i in range(length):
                if self.excel_data.loc[i,'Activation'] == 1: # a new order detected
                    row           = self.excel_data.iloc[i]
                    self.symbol        = 'N225M'
                    self.exchange      = 'OSE.JPN' 
                    self.trigger_level = row['Trigger_Level_High_Low']
                    self.entry_type    = row['Entry_Type']
                    self.entry_strike  = row['Entry_Strike']
                    self.strike_type   = row['Strike_Type']
                    self.expiry        = str(row['Expiry'])
                    self.target        = row['Target']
                    self.stop_loss     = row['Stop_Loss']
                    self.qty           = row['Qty']
                    self.slicing       = row['Slicing']
                    self.time_interval = row['Time_Interval']
                    self.activation    = row['Activation']
                    # if self.strike_type == 'CE': # CHANGED THIS LINE
                    if self.strike_type == "SELL":
                        self.side = 'SELL'
                    else:
                        self.side = 'BUY' 

                    datevar = self.expiry
                    date,timep = datevar.split(' ') # changed
                    # date = datevar
                    print(date)
                    year,day,month = date.split('-')
                    print(f"year is {year}")
                    print(f"month is {month}")
                    print(f"day is {day}")
                    # day,month,year = date.split('-')
                    # formatted_date = f"{year}{month.zfill(2)}" # changed
                    formatted_date = f"{year}{month.zfill(2)}{day}"
                    
                    print(f"formatted_date is {formatted_date}")
                    # exit(0)
                    contract = Future(symbol=self.symbol,exchange=self.exchange,lastTradeDateOrContractMonth=str(formatted_date))
                    print(self.trigger_level)
                    print(await self.get_current_market_price_futures(contract)) 
                    print(self.entry_type)
                    print(self.strike_type)
                    # if self.strike_type == "PE" and self.trigger_level < await self.get_current_market_price_futures(contract):
                    if self.strike_type == "BUY" and self.trigger_level <= await self.get_current_market_price_futures(contract): # current_price breaks through trigger_level
                        if self.entry_type == "LIMIT":
                            for _ in range(0,int(self.qty/self.slicing),1):
                                datevar = self.expiry
                                date,timep = datevar.split(' ')
                                # date = datevar 
                                print(f"date is {date}")
                                # year,day,month = date.split('-')
                                # formatted_date = f"{year}{month.zfill(2)}"
                                formatted_date = f"{year}{month.zfill(2)}{day}"
                                print(f"formatted_date is {formatted_date}")
                                # exit(0)
                                self.contract       = Future(symbol=self.symbol,exchange=self.exchange,lastTradeDateOrContractMonth=str(formatted_date))
                                bid,ask = await self.get_bid_and_ask(contractmonth=formatted_date)
                                attempt = 0
                                # while attempt<3:
                                # successful_orders = 0
                                # failed_orders = 0
                                while attempt<int(credentials.attempts):
                                    if credentials.trade_type_default == 0:
                                        self.order         = LimitOrder(action=self.side,totalQuantity=str(int(self.slicing)),lmtPrice=str(self.entry_strike)) 
                                    else:
                                        print(f"using trade_type default {credentials.trade_type_default}")
                                        self.order         = LimitOrder(action=self.side,totalQuantity=str(int(self.slicing)),lmtPrice=str(int((bid + (2**attempt - 1)*ask)/2**attempt))) 
                                    self.order.account = 'DU9727656'
                                    self.order.transmit = True
                                    print(f"Placing limit order,attempt {attempt+1}")
                                    self.order_details = self.client.placeOrder(contract=self.contract,order=self.order)
                                    print(self.order_details)
                                    await asyncio.sleep(2) # keep this part same
                                    print(self.order_details.isDone())

                                    if not self.order_details.isDone():
                                        print("The cancelled order is :\n")
                                        self.canceled_order_details = self.client.cancelOrder(order=self.order_details.orderStatus)
                                        print(self.canceled_order_details)
                                        print("Order failed")
                                        # failed_orders = failed_orders+1
                                    else:
                                        print("Limit order placed successfully")
                                        print(self.order_details)
                                        # successful_orders = successful_orders+1
                                        break
                                    
                                    attempt = attempt+1
                                
                                # if attempt == 3:
                                if attempt == credentials.attempts:
                                    print(f"Limit order failed {credentials.attempts} times placing market order")
                                    self.order = MarketOrder(action=self.side,totalQuantity=str(int(self.slicing)))
                                    self.order_details = self.client.placeOrder(contract=self.contract,order=self.order)
                                    print("Market order placed")
                                    print(self.order_details)

                            self.excel_data.loc[i,'Activation'] = -1 
                            with pd.ExcelWriter(self.path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
                                self.excel_data.to_excel(writer, sheet_name="Sheet1", index=False)
                            if credentials.user_time_default == 0:
                                await asyncio.sleep(self.time_interval)
                            else:
                                print(f"sleeping for {credentials.user_time}")
                                await asyncio.sleep(credentials.user_time)

                        else:
                            for _ in range(0,int(self.qty/self.slicing),1):
                                datevar = self.expiry
                                date,timep = datevar.split(' ')
                                # date = datevar
                                # year,day,month = date.split('-')
                                print(date)
                                day,month,year = date.split('-')
                                formatted_date = f"{year}{month.zfill(2)}"
                                # formatted_date = f"{year}{month.zfill(2)}{day}"
                                print(f"formatted_date is {formatted_date}")
                                # exit(0)
                                self.contract       = Future(symbol=self.symbol,exchange=self.exchange,lastTradeDateOrContractMonth=str(formatted_date))
                                self.order          = MarketOrder(action=self.side,totalQuantity=str(int(self.slicing))) 
                                self.order.account = 'DU9727656'
                                self.order.transmit = True
                                self.order_details = self.client.placeOrder(contract=self.contract,order=self.order)
                                # await asyncio.sleep(3)
                                if credentials.user_time_default == 0:
                                    await asyncio.sleep(self.time_interval)
                                else:
                                    print(f"sleep for {credentials.user_time}")
                                    await asyncio.sleep(credentials.user_time)
                                print("The order has been placed")
                            self.excel_data.loc[i,'Activation'] = -1 
                            with pd.ExcelWriter(self.path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
                                self.excel_data.to_excel(writer, sheet_name="Sheet1", index=False)
                            if credentials.user_time_default == 0:
                                await asyncio.sleep(self.time_interval)
                            else:
                                print(f"sleeping for {credentials.user_time}")
                                await asyncio.sleep(credentials.user_time)

                    elif self.strike_type == "SELL" and self.trigger_level >= await self.get_current_market_price_futures(contract):
                        if self.entry_type == "LIMIT":
                            for _ in range(0,int(self.qty/self.slicing),1):
                                datevar = self.expiry
                                date,timep = datevar.split(' ')
                                # date = datevar
                                # year,day,month = date.split('-')
                                print(date)
                                day,month,year = date.split('-')
                                formatted_date = f"{year}{month.zfill(2)}"
                                # formatted_date = f"{year}{month.zfill(2)}{day}"
                                print(f"formatted_date is {formatted_date}")
                                # exit(0)
                                self.contract       = Future(symbol=self.symbol,exchange=self.exchange,lastTradeDateOrContractMonth=str(formatted_date))
                                bid,ask = await self.get_bid_and_ask(contractmonth=formatted_date)
                                attempt = 0
                                while attempt<int(credentials.attempts):
                                    if credentials.trade_type_default == 0:
                                        self.order         = LimitOrder(action=self.side,totalQuantity=str(int(self.slicing)),lmtPrice=str(self.entry_strike))
                                    else:
                                        print(f"using trade_type default {credentials.trade_type_default}")
                                        self.order         = LimitOrder(action=self.side,totalQuantity=str(int(self.slicing)),lmtPrice=str(int((bid + (2**attempt - 1)*ask)/2**attempt)))  
                                    self.order.account = 'DU9727656'
                                    self.order.transmit = True
                                    print(f"Placing limit order,attempt {attempt+1}")
                                    self.order_details = self.client.placeOrder(contract=self.contract,order=self.order)
                                    print(self.order_details)
                                    await asyncio.sleep(2) # keep this same
                                    print(self.order_details.isDone())

                                    if not self.order_details.isDone():
                                        print("The cancelled order is :\n")
                                        self.canceled_order_details = self.client.cancelOrder(order=self.order_details.orderStatus)
                                        print(self.canceled_order_details)
                                        print("Order failed")
                                    else:
                                        print("Limit order placed successfully")
                                        print(self.order_details)
                                        break
                                    attempt = attempt+1
                                
                                # if attempt == 3:
                                if attempt == credentials.attempts:
                                    print(f"Limit order failed {credentials.attempts} times placing market order")
                                    self.order = MarketOrder(action=self.side,totalQuantity=str(int(self.slicing)))
                                    self.order_details = self.client.placeOrder(contract=self.contract,order=self.order)
                                    print("Market order placed")
                                    print(self.order_details)

                            self.excel_data.loc[i,'Activation'] = -1 # now use excelwriter fn
                            with pd.ExcelWriter(self.path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
                                self.excel_data.to_excel(writer, sheet_name="Sheet1", index=False)
                            if credentials.user_time_default == 0:
                                await asyncio.sleep(self.time_interval)
                            else:
                                print(f"sleeping for {credentials.user_time}")
                                await asyncio.sleep(credentials.user_time)

                        else:
                            for _ in range(0,int(self.qty/self.slicing),1):
                                datevar = self.expiry
                                date,timep = datevar.split(' ')
                                # date = datevar
                                print(date)
                                # year,day,month = date.split('-')
                                day,month,year = date.split('-')
                                formatted_date = f"{year}{month.zfill(2)}"
                                # formatted_date = f"{year}{month.zfill(2)}{day}"
                                print(f"formatted_date is {formatted_date}")
                                # exit(0)
                                self.contract       = Future(symbol=self.symbol,exchange=self.exchange,lastTradeDateOrContractMonth=str(formatted_date))
                                self.order          = MarketOrder(action=self.side,totalQuantity=str(int(self.slicing))) 
                                self.order.account = 'DU9727656'
                                self.order.transmit = True
                                self.order_details = self.client.placeOrder(contract=self.contract,order=self.order)
                                await asyncio.sleep(3) # keep this same
                                print(self.order_details)
                                print("The order has been placed")
                            self.excel_data.loc[i,'Activation'] = -1 # now use excelwriter fn
                            with pd.ExcelWriter(self.path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
                                self.excel_data.to_excel(writer, sheet_name="Sheet1", index=False)
                            if credentials.user_time_default == 0:
                                await asyncio.sleep(self.time_interval)
                            else:
                                print(f"sleeping for {credentials.user_time}")
                                await asyncio.sleep(credentials.user_time)
                    else:
                        print("The trigger price has not being triggered")
        else:
            print("No changes in excel")

    async def close_empty_trigger_fn(self):
        print("fn in close_all_if_trigger")
        self.df = pd.read_excel(self.path, sheet_name="Sheet1") 
        df = self.df
        self.expiryvar = None
        for i in range(len(self.df)):
            if(self.df.loc[i,'Target'] == "-" and self.df.loc[i,'Stop_Loss'] == "-" and self.df.loc[i,'Strike_Type'] == "SELL" and self.df.loc[i,'Activation'] == 2): # have the closing only if the activation is one 
                print(f"lower trigger limit at index : {i} at {self.df.loc[i,'Entry_Strike']}")
                self.lower_trigger = self.df.loc[i,'Entry_Strike']
                self.expiryvar       = self.df.loc[i,'Expiry']
                break # close one kind of expiry at a time.

            elif(self.df.loc[i,'Target'] == "-" and self.df.loc[i,'Stop_Loss'] == "-" and self.df.loc[i,'Strike_Type'] == "BUY" and self.df.loc[i,'Activation'] == 2):
                print(f"upper trigger limit at index : {i} at {self.df.loc[i,'Entry_Strike']}")
                self.upper_trigger = self.df.loc[i,'Entry_Strike']
                self.expiryvar       = self.df.loc[i,'Expiry']
                break # close one kind of expiry at a time.

        if self.expiryvar is not None:
            # print(f"closing trades for {self.expiryvar}")
            datevar = self.expiryvar
            # print(f"datevar in line 296 is {datevar}")
            datevar_str = datevar.strftime('%Y-%m-%d') if isinstance(datevar, pd.Timestamp) else str(datevar)
            year,day,month = datevar_str.split('-')
            # print(f"year is {year}")
            # print(f"day is {day}")
            # print(f"month is {month}")
            # formatted_date = f"{year}{month.zfill(2)}"
            formatted_date = f"{year}{month.zfill(2)}{day}"
            # print(f"formatted_date in outer loop is {formatted_date}")
            contract = Future(symbol=self.symbol,exchange=self.exchange,lastTradeDateOrContractMonth=str(formatted_date))

            price = await self.get_current_market_price_futures(contract)
            positions = self.client.positions()
            print(f"current price is {price}")
            if price>self.upper_trigger or price<self.lower_trigger:
                if positions:
                    for i in range(len(df)):
                        if self.df.loc[i,'Activation'] == -1 and self.df.loc[i,'Expiry'] == self.expiryvar:
                            # print(f"Activation is {self.df.loc[i,'Activation']}")
                            # print(f"Expiry is {self.expiryvar}")
                            # print(f"Closing position of type {formatted_date} at index {i}")
                            # print(f"formatted date in inner loop is : {formatted_date}")
                            contract = Future(symbol='N225M', exchange='OSE.JPN', lastTradeDateOrContractMonth=str(formatted_date))
                            if self.df.loc[i, 'Strike_Type'] == 'SELL':
                                current_action = 'BUY'
                            else:
                                current_action = 'SELL'
                            order = MarketOrder(action=current_action, totalQuantity=self.df.loc[i, 'Qty'])
                            order.account = 'DU9727656'
                            order.transmit = True
                            result = self.client.placeOrder(contract, order)
                            self.df.loc[i, 'Activation'] = 0
                else:
                    print("Positions are empty")

            with pd.ExcelWriter(self.path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
                self.df.to_excel(writer, sheet_name="Sheet1", index=False)
        else:
            print("No condition for closing till yet")
    

    async def get_current_market_price_futures(self, contract): 
        """
        Fetches the current market price of the given futures contract.
        """
        self.client.reqMarketDataType(3)
        ticker = self.client.reqMktData(contract, '', False, False)
        self.client.sleep(1)  
        if ticker.last is not None: 
            return ticker.last
        if ticker.close is not None: 
            return ticker.close
        
        print(ticker.last)
        return None
        # return 0.0
    
    async def show_details(self):
        result = self.ib.reqOpenOrders()
        return result
    
    async def get_bid_and_ask(self,contractmonth):
        self.client.reqMarketDataType(3)
        contract = Future(symbol=self.symbol,exchange=self.exchange,lastTradeDateOrContractMonth=contractmonth)
        test = self.ib.reqTickers(contract)
        for _,r in enumerate(test):
            bid,ask = r.bid,r.ask
            print(bid)
            print(ask)
        
        return bid,ask
    
    async def check_for_tp_sl(self, current_price, target_price,stop_loss,action): 
        if action == 'BUY':
            if current_price >= target_price:  # Target Price hit
                return "SELL"
            elif current_price <= stop_loss:  # Stop Loss hit
                return "SELL"

        elif action == 'SELL':
            if current_price <= target_price:  # corrected
                return "BUY"
            elif current_price >= stop_loss:  # corrected
                return "BUY"
        return None

    async def monitor_tp_sl(self): 
        self.df = pd.read_excel(self.path, sheet_name="Sheet1")  
        for i in range(len(self.df)):
            if self.df.loc[i,'Activation'] == -1 and self.df.loc[i,'Strike_Type'] == 'BUY':
                datevar = self.df.loc[i, 'Expiry']
                # Ensure datevar is a string in 'YYYY-MM-DD' format
                datevar_str = datevar.strftime('%Y-%m-%d') if isinstance(datevar, pd.Timestamp) else str(datevar)
                year,day,month = datevar_str.split('-')  # Ensure the date is in 'YYYY-MM-DD HH:MM:SS' format
                formatted_date = f"{year}{month.zfill(2)}"
                # formatted_date = f"{year}{month.zfill(2)}{day}"
                contract      = Future(symbol='N225M',exchange='OSE.JPN',lastTradeDateOrContractMonth=str(formatted_date))
                current_price = await self.get_current_market_price_futures(contract)
                if current_price:
                    action = await self.check_for_tp_sl(current_price, self.df['Target'].iloc[i],self.df['Stop_Loss'].iloc[i],self.df.loc[i,'Strike_Type'])
                    if action is not None:  
                        print(f"An action of sell has been triggered in row {i}")
                        order = MarketOrder(action='SELL', totalQuantity=str(self.df['Qty'].iloc[i]))
                        order.account = 'DU9727656'
                        order.transmit = True
                        result = self.client.placeOrder(contract, order)
                        print(result)
                        self.df.loc[i, 'Activation'] = 0
                        print("One position is being closed")  
                    else:
                        print("No profit/loss is triggered")

            elif self.df.loc[i,'Activation'] == -1 and self.df.loc[i,'Strike_Type'] == 'SELL':
                datevar = self.df.loc[i, 'Expiry']
                print(datevar)
                # Ensure datevar is a string in 'YYYY-MM-DD' format
                datevar_str = datevar.strftime('%Y-%m-%d') if isinstance(datevar, pd.Timestamp) else str(datevar)
                year,day,month = datevar_str.split('-')  # Ensure the date is in 'YYYY-MM-DD HH:MM:SS' format
                formatted_date = f"{year}{month.zfill(2)}" 
                # formatted_date = f"{year}{month.zfill(2)}{day}"

                contract      = Future(symbol='N225M',exchange='OSE.JPN',lastTradeDateOrContractMonth=str(formatted_date)) # change this line
                current_price = await self.get_current_market_price_futures(contract)
                if current_price:
                    action = await self.check_for_tp_sl(current_price, self.df['Target'].iloc[i],self.df['Stop_Loss'].iloc[i],self.df.loc[i,'Strike_Type'])
                    if action is not None:  
                        print(f"An action of buy has been triggered in row {i}")
                        order = MarketOrder(action='BUY', totalQuantity=str(self.df['Qty'].iloc[i]))
                        order.account = 'DU9727656'
                        order.transmit = True
                        result = self.client.placeOrder(contract, order)
                        print(result)
                        self.df.loc[i, 'Activation'] = 0  
                    else:
                        print("No profit/loss is triggered")

        with pd.ExcelWriter(self.path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
            self.df.to_excel(writer, sheet_name="Sheet1", index=False)
    
    async def new_auto_square_off(self): # put this in async
        self.df = pd.read_excel(self.path, sheet_name="Sheet1")
        df = self.df
        current_time = datetime.now().strftime("%H:%M")
        positions = self.client.positions()
        # if current_time > "9:10":
        if current_time >= str(credentials.current_time):
            if positions:
                for i in range(len(df)):
                    if self.df.loc[i,'Activation'] == -1:
                        datevar = self.df.loc[i, 'Expiry']
                        # Ensure datevar is a string in 'YYYY-MM-DD' format
                        datevar_str = datevar.strftime('%Y-%m-%d') if isinstance(datevar, pd.Timestamp) else str(datevar)
                        year,day,month = datevar_str.split('-')  # Ensure the date is in 'YYYY-MM-DD HH:MM:SS' format
                        formatted_date = f"{year}{month.zfill(2)}" # changed
                        # formatted_date = f"{year}{month.zfill(2)}{day}"
                        contract = Future(symbol='N225M', exchange='OSE.JPN', lastTradeDateOrContractMonth=str(formatted_date))
                        if self.df.loc[i, 'Strike_Type'] == 'SELL':
                            current_action = 'BUY'
                        else:
                            current_action = 'SELL'
                        order = MarketOrder(action=current_action, totalQuantity=self.df.loc[i, 'Qty'])
                        order.account = 'DU9727656'
                        order.transmit = True
                        result = self.client.placeOrder(contract, order)
                        self.df.loc[i, 'Activation'] = 0
            else:
                print("Positions are empty")
        else:
            print("The time is not for closing the market is not yet")

        with pd.ExcelWriter(self.path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
            self.df.to_excel(writer, sheet_name="Sheet1", index=False)

    async def run(self):
        print("The process has started")
        await self.connection_show()
        while True:
            # await asyncio.gather(self.check_for_new_positions(),self.new_auto_square_off(),self.monitor_tp_sl())
            await asyncio.gather(self.check_for_new_positions(),self.new_auto_square_off(),self.monitor_tp_sl(),self.close_empty_trigger_fn())
            await asyncio.sleep(10) # keep this same

if __name__ == "__main__":
    if credentials.master is not False:
        session = IBRKExcel()
        asyncio.run(session.run())
    else:
        print("The bot is currently off make changes in the master.")

# testing 

# 1 - check new_auto_square_off and tp/sl - working
# 2 - check close_empty_trigger_fn - 
# 3 - check placing above/below buy/sell orders  

# latest code saved on 6th March 2025

In [ ]:
# backup - 3/18/25 some changes were made regarding reducing the redundancy / time but the code should remain correct 

import time
import asyncio
import pandas as pd
from datetime import datetime,timedelta
from ib_broker import *
import credentials
from openpyxl import load_workbook
import nest_asyncio

nest_asyncio.apply()

class IBRKExcel:
    def __init__(self):
        self.symbol          = credentials.symbol
        self.exchange        = credentials.exchange
        self.path            = credentials.xlsx_path_1
        self.current_time    = datetime.now()
        self.excel_data      = pd.read_excel(self.path, sheet_name=credentials.sheet_name) 
        self.length          = len(self.excel_data)
        self.upper_trigger   = 100000000
        self.lower_trigger   = -100000000
        self.current_time    = datetime.now().time()

    async def check_excel_changes(self):
        new_data   = pd.read_excel(self.path, sheet_name=credentials.sheet_name)
        new_length = len(new_data)

        if new_length  != self.length:
            self.length     = new_length
            self.excel_data = new_data
            return True
        return False

    async def connection_show(self) -> bool:
        host, port = credentials.host, credentials.port
        self.client = IB()
        self.ib = self.client
        # connection_print = self.client.connect(host=host,port=port,clientId=13,account='DU9727656',timeout=60)
        connection_print = self.client.connect(host=host,port=port,clientId=13,account=credentials.account_no,timeout=60)
        print(connection_print)

    async def check_for_new_positions(self): # put this in async
        if await self.check_excel_changes():
            print("a change on the excel has been made")
            length   = len(pd.read_excel(self.path, sheet_name=credentials.sheet_name))

            for i in range(length):
                if self.excel_data.loc[i,'Activation'] == 1: # a new order detected
                    row           = self.excel_data.iloc[i]
                    self.symbol             = 'N225M'
                    self.exchange           = 'OSE.JPN' 
                    self.trigger_level      = row['Trigger_Level_High_Low']
                    self.entry_type         = row['Entry_Type']
                    self.entry_strike       = row['Entry_Strike']
                    self.option_strike_type = row['Option_Type']
                    self.strike_type        = row['Strike_Type']
                    self.expiry             = str(row['Expiry'])
                    self.target             = row['Target']
                    self.stop_loss          = row['Stop_Loss']
                    self.qty                = row['Qty']
                    self.slicing            = row['Slicing']
                    self.time_interval      = row['Time_Interval']
                    self.activation_type    = row['Activation_Type']
                    self.activation         = row['Activation']

                    if self.strike_type == "SELL":
                        self.side = 'SELL'
                    else:
                        self.side = 'BUY' 

                    datevar = self.expiry
                    datevar = datevar.strftime('%Y-%m-%d') if isinstance(datevar, pd.Timestamp) else str(datevar)
                    date,timep = datevar.split(' ') 
                    year,day,month = date.split('-')
                    print(f"year is {year}")
                    print(f"month is {month}")
                    print(f"day is {day}")
                    formatted_date = f"{year}{month.zfill(2)}{day}"
                    
                    print(f"formatted_date is {formatted_date}")
                    contract = Future(symbol=self.symbol,exchange=self.exchange,lastTradeDateOrContractMonth=str(formatted_date))
                    print(self.trigger_level)
                    print(await self.get_current_market_price_futures(contract)) 
                    print(self.entry_type)
                    print(self.strike_type)

                    if self.strike_type == "BUY" and self.trigger_level <= await self.get_current_market_price_futures(contract): # current_price breaks through trigger_level
                    # if self.strike_type == "BUY":
                        if self.entry_type == "LIMIT":
                            for _ in range(0,int(self.qty/self.slicing),1):
                                datevar = self.expiry
                                datevar = datevar.strftime('%Y-%m-%d') if isinstance(datevar, pd.Timestamp) else str(datevar)
                                date,timep = datevar.split(' ')
                                print(f"date is {date}")
                                formatted_date = f"{year}{month.zfill(2)}{day}"
                                print(f"formatted_date is {formatted_date}")
                                self.contract = Option(symbol=self.symbol,exchange=self.exchange,lastTradeDateOrContractMonth=str(formatted_date),strike=float(self.entry_strike),right=self.option_strike_type)
                                bid,ask = await self.get_bid_and_ask_options(contractmonth=formatted_date,entry_strike=self.entry_strike,option_strike_type=self.option_strike_type)
                                print(f"the bid is {bid} and the ask is {ask}")
                                attempt = 0

                                while attempt<int(credentials.attempts):
                                    print(f"the current bid is {bid} and the current ask is {ask} and the order is being placed at {float(str(int((bid + (2**attempt - 1)*ask)/2**attempt)))}.")
                                    # if credentials.trade_type_default == 0:
                                    #     self.order         = LimitOrder(action=self.side,totalQuantity=str(int(self.slicing)),lmtPrice=str(self.entry_strike)) 
                                    # else:
                                    #     print(f"using trade_type default {credentials.trade_type_default}")
                                    #     self.order         = LimitOrder(action=self.side,totalQuantity=str(int(self.slicing)),lmtPrice=str(int((bid + (2**attempt - 1)*ask)/2**attempt))) 
                                    # self.order.account = 'DU9727656'
                                    self.order         = LimitOrder(action=self.side,totalQuantity=str(int(self.slicing)),lmtPrice=str(int((bid + (2**attempt - 1)*ask)/2**attempt))) 
                                    self.order.account = credentials.account_no
                                    self.order.transmit = True
                                    print(f"Placing limit order,attempt {attempt+1}")
                                    self.order_details = self.client.placeOrder(contract=self.contract,order=self.order)
                                    print(self.order_details)
                                    await asyncio.sleep(credentials.pause_time) # sleep before isDone
                                    print(self.order_details.isDone())

                                    if not self.order_details.isDone():
                                        print("The cancelled order is :\n")
                                        self.canceled_order_details = self.client.cancelOrder(order=self.order_details.orderStatus)
                                        print(self.canceled_order_details)
                                        print("Order failed")
                                    else:
                                        print("Limit order placed successfully")
                                        print(self.order_details)
                                        break
                                    
                                    attempt = attempt+1
                                    await asyncio.sleep(self.time_interval)

                                if attempt == credentials.attempts:
                                    print(f"Limit order failed {credentials.attempts} times placing market order")
                                    self.order = MarketOrder(action=self.side,totalQuantity=str(int(self.slicing)))
                                    self.order_details = self.client.placeOrder(contract=self.contract,order=self.order)
                                    print("Market order placed")
                                    print(self.order_details)

                            self.excel_data.loc[i,'Activation'] = -1 
                            with pd.ExcelWriter(self.path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
                                self.excel_data.to_excel(writer, sheet_name=credentials.sheet_name, index=False)
                            
                            # await asyncio.sleep(self.time_interval)
                            # if credentials.user_time_default == 0:
                            #     await asyncio.sleep(self.time_interval) # sleep at the end and placing and order
                            # else:
                            #     print(f"sleeping for {credentials.user_time}")
                            #     await asyncio.sleep(credentials.user_time) # sleep at the end and placing an order

                        else:
                            for _ in range(0,int(self.qty/self.slicing),1):
                                datevar = self.expiry
                                datevar = datevar.strftime('%Y-%m-%d') if isinstance(datevar, pd.Timestamp) else str(datevar)
                                date,timep = datevar.split(' ')
                                year,day,month = date.split('-')
                                formatted_date = f"{year}{month.zfill(2)}{day}"
                                print(f"formatted_date is {formatted_date}")
                                self.contract = Option(symbol=self.symbol,exchange=self.exchange,lastTradeDateOrContractMonth=str(formatted_date),strike=float(self.entry_strike),right=self.option_strike_type)
                                self.order          = MarketOrder(action=self.side,totalQuantity=str(int(self.slicing))) 
                                # self.order.account = 'DU9727656'
                                self.order.account = credentials.account_no
                                self.order.transmit = True
                                self.order_details = self.client.placeOrder(contract=self.contract,order=self.order)
                                # await asyncio.sleep(self.time_interval)
                                # if credentials.user_time_default == 0:
                                #     await asyncio.sleep(self.time_interval) # sleep at the end of placing an order
                                # else:
                                #     print(f"sleep for {credentials.user_time}")
                                #     await asyncio.sleep(credentials.user_time) # sleep at the end of placing an order
                                print("The order has been placed")
                                await asyncio.sleep(self.time_interval)
                            self.excel_data.loc[i,'Activation'] = -1 
                            with pd.ExcelWriter(self.path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
                                self.excel_data.to_excel(writer, sheet_name=credentials.sheet_name, index=False)

                            # await asyncio.sleep(self.time_interval)
                            # if credentials.user_time_default == 0:
                            #     await asyncio.sleep(self.time_interval) # sleep after the order is done
                            # else:
                            #     print(f"sleeping for {credentials.user_time}")
                            #     await asyncio.sleep(credentials.user_time) # sleep after the order is done 

                    elif self.strike_type == "SELL" and self.trigger_level >= await self.get_current_market_price_futures(contract):
                    # elif self.strike_type == "SELL":
                        if self.entry_type == "LIMIT":

                            for _ in range(0,int(self.qty/self.slicing),1):
                                datevar = self.expiry
                                datevar = datevar.strftime('%Y-%m-%d') if isinstance(datevar, pd.Timestamp) else str(datevar)
                                date,timep = datevar.split(' ')
                                year,day,month = date.split('-')
                                formatted_date = f"{year}{month.zfill(2)}{day}"
                                print(f"formatted_date is {formatted_date}")
                                self.contract = Option(symbol=self.symbol,exchange=self.exchange,lastTradeDateOrContractMonth=str(formatted_date),strike=float(self.entry_strike),right=self.option_strike_type)
                                bid,ask = await self.get_bid_and_ask_options(contractmonth=formatted_date,entry_strike=self.entry_strike,option_strike_type=self.option_strike_type)
                                attempt = 0

                                while attempt<int(credentials.attempts):
                                    # if credentials.trade_type_default == 0:
                                    #     self.order         = LimitOrder(action=self.side,totalQuantity=str(int(self.slicing)),lmtPrice=str(self.entry_strike))
                                    # else:
                                    #     print(f"using trade_type default {credentials.trade_type_default}")
                                    #     self.order         = LimitOrder(action=self.side,totalQuantity=str(int(self.slicing)),lmtPrice=str(int((bid + (2**attempt - 1)*ask)/2**attempt)))  
                                    # self.order.account = 'DU9727656'
                                    self.order         = LimitOrder(action=self.side,totalQuantity=str(int(self.slicing)),lmtPrice=str(int((bid + (2**attempt - 1)*ask)/2**attempt)))
                                    self.order.account = credentials.account_no
                                    self.order.transmit = True
                                    print(f"Placing limit order,attempt {attempt+1}")
                                    self.order_details = self.client.placeOrder(contract=self.contract,order=self.order)
                                    print(self.order_details)
                                    await asyncio.sleep(credentials.pause_time) # sleep to check whether the order is done or not
                                    print(self.order_details.isDone())

                                    if not self.order_details.isDone():
                                        print("The cancelled order is :\n")
                                        self.canceled_order_details = self.client.cancelOrder(order=self.order_details.orderStatus)
                                        print(self.canceled_order_details)
                                        print("Order failed")
                                    else:
                                        print("Limit order placed successfully")
                                        print(self.order_details)
                                        break
                                    attempt = attempt+1
                                    await asyncio.sleep(self.time_interval)

                                if attempt == credentials.attempts:
                                    print(f"Limit order failed {credentials.attempts} times placing market order")
                                    self.order = MarketOrder(action=self.side,totalQuantity=str(int(self.slicing)))
                                    self.order_details = self.client.placeOrder(contract=self.contract,order=self.order)
                                    print("Market order placed")
                                    print(self.order_details)

                            self.excel_data.loc[i,'Activation'] = -1 # now use excelwriter fn
                            with pd.ExcelWriter(self.path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
                                self.excel_data.to_excel(writer, sheet_name=credentials.sheet_name, index=False)

                            # await asyncio.sleep(self.time_interval)
                            # if credentials.user_time_default == 0:
                            #     await asyncio.sleep(self.time_interval) # sleep after the order has been placed
                            # else:
                            #     print(f"sleeping for {credentials.user_time}")
                            #     await asyncio.sleep(credentials.user_time) # sleep after the order has been placed

                        else:
                            for _ in range(0,int(self.qty/self.slicing),1):
                                datevar = self.expiry
                                datevar = datevar.strftime('%Y-%m-%d') if isinstance(datevar, pd.Timestamp) else str(datevar)
                                date,timep = datevar.split(' ')
                                year,day,month = date.split('-')
                                formatted_date = f"{year}{month.zfill(2)}{day}"
                                print(f"formatted_date is {formatted_date}")
                                self.contract = Option(symbol=self.symbol,exchange=self.exchange,lastTradeDateOrContractMonth=str(formatted_date),strike=float(self.entry_strike),right=self.option_strike_type)
                                self.order          = MarketOrder(action=self.side,totalQuantity=str(int(self.slicing))) 
                                # self.order.account = 'DU9727656'
                                self.order.account = credentials.account_no
                                self.order.transmit = True
                                self.order_details = self.client.placeOrder(contract=self.contract,order=self.order)
                                await asyncio.sleep(credentials.pause_time)  # sleep before showing order details
                                print(self.order_details)
                                print("The order has been placed")
                                await asyncio.sleep(self.time_interval)

                            self.excel_data.loc[i,'Activation'] = -1 
                            with pd.ExcelWriter(self.path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
                                self.excel_data.to_excel(writer, sheet_name=credentials.sheet_name, index=False)

                            # if credentials.user_time_default == 0:
                            #     await asyncio.sleep(self.time_interval) # sleep after the order has been placed
                            # else:
                            #     print(f"sleeping for {credentials.user_time}")
                            #     await asyncio.sleep(credentials.user_time) # sleep after the order has been placed
                            # await asyncio.sleep(self.time_interval)
                    else:
                        print("The trigger price has not being triggered")
                
    async def close_empty_trigger_fn_upper(self):
        print("fn in close_all_if_trigger upper")
        self.df = pd.read_excel(self.path, sheet_name=credentials.sheet_name) 
        df = self.df
        self.expiryvar = None

        for i in range(len(self.df)):
            if(self.df.loc[i,'Target'] == "-" and self.df.loc[i,'Stop_Loss'] == "-" and self.df.loc[i,'Strike_Type'] == "BUY" and self.df.loc[i,'Activation'] == 1 and self.df.loc[i,'Activation_Type'] == 2):
                print(f"upper trigger limit at index : {i} at {self.df.loc[i,'Entry_Strike']}")
                datevar = self.df.loc[i,'Expiry']
                datevar_str = datevar.strftime('%Y-%m-%d') if isinstance(datevar, pd.Timestamp) else str(datevar)
                year,day,month = datevar_str.split('-')
                formatted_date = f"{year}{month.zfill(2)}{day}"
                contract = Future(symbol=self.symbol,exchange=self.exchange,lastTradeDateOrContractMonth=str(formatted_date))

                price = await self.get_current_market_price_futures(contract)
                positions = self.client.positions()
                print(f"current price is {price}")
                if price>self.upper_trigger:
                    if positions:
                        for i in range(len(df)):
                            if self.df.loc[i,'Activation'] == -1 and self.df.loc[i,'Expiry'] == self.expiryvar:
                                datevar = self.df.loc[i,'Expiry']
                                datevar_str = datevar.strftime('%Y-%m-%d') if isinstance(datevar, pd.Timestamp) else str(datevar)
                                year,day,month = datevar_str.split('-')
                                formatted_date = f"{year}{month.zfill(2)}{day}"
                                self.option_strike_type = self.df.loc[i,'Option_Type']
                                self.entry_strike = self.df.loc[i,'Entry_Strike']
                                contract = Option(symbol=self.symbol,exchange=self.exchange,lastTradeDateOrContractMonth=str(formatted_date),strike=float(self.entry_strike),right=self.option_strike_type)
                                if self.df.loc[i, 'Strike_Type'] == 'SELL':
                                    current_action = 'BUY'
                                else:
                                    current_action = 'SELL'
                                order = MarketOrder(action=current_action, totalQuantity=self.df.loc[i, 'Qty'])
                                # order.account = 'DU9727656'
                                order.account = credentials.account_no
                                order.transmit = True
                                result = self.client.placeOrder(contract, order)
                                self.df.loc[i, 'Activation'] = 0
                                
                            elif self.df.loc[i,'Activation'] == 3 and self.df.loc[i,'Expiry'] == self.expiryvar:
                                datevar = self.df.loc[i,'Expiry']
                                datevar_str = datevar.strftime('%Y-%m-%d') if isinstance(datevar, pd.Timestamp) else str(datevar)
                                year,day,month = datevar_str.split('-')
                                formatted_date = f"{year}{month.zfill(2)}{day}"
                                self.option_strike_type = self.df.loc[i,'Option_Type']
                                self.entry_strike = self.df.loc[i,'Entry_Strike']
                                contract = Option(symbol=self.symbol,exchange=self.exchange,lastTradeDateOrContractMonth=str(formatted_date),strike=float(self.entry_strike),right=self.option_strike_type)
                                if self.df.loc[i, 'Strike_Type'] == 'SELL':
                                    current_action = 'BUY'
                                else:
                                    current_action = 'SELL'
                                order = MarketOrder(action=current_action, totalQuantity=self.df.loc[i, 'Qty'])
                                # order.account = 'DU9727656'
                                order.account = credentials.account_no
                                order.transmit = True
                                result = self.client.placeOrder(contract, order)
                                self.df.loc[i, 'Activation'] = 0
                else:
                    print("Positions are empty")

            with pd.ExcelWriter(self.path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
                self.df.to_excel(writer, sheet_name=credentials.sheet_name, index=False)
        else:
            print("No condition for closing till yet")

    async def close_empty_trigger_fn_lower(self):
        print("fn in close_all_if_trigger lower")
        self.df = pd.read_excel(self.path, sheet_name=credentials.sheet_name) 
        df = self.df
        self.expiryvar = None

        for i in range(len(self.df)):
            if(self.df.loc[i,'Target'] == "-" and self.df.loc[i,'Stop_Loss'] == "-" and self.df.loc[i,'Strike_Type'] == "SELL" and self.df.loc[i,'Activation_Type'] == 2 and self.df.loc[i,'Activation'] == 1): # have the closing only if the activation is one 
                print(f"lower trigger limit at index : {i} at {self.df.loc[i,'Entry_Strike']}")
                datevar = self.df.loc[i,'Expiry']
                datevar_str = datevar.strftime('%Y-%m-%d') if isinstance(datevar, pd.Timestamp) else str(datevar)
                year,day,month = datevar_str.split('-')
                formatted_date = f"{year}{month.zfill(2)}{day}"
                contract = Future(symbol=self.symbol,exchange=self.exchange,lastTradeDateOrContractMonth=str(formatted_date))

                price = await self.get_current_market_price_futures(contract)
                positions = self.client.positions()
                print(f"current price is {price}")

                if price<self.lower_trigger:
                    if positions:
                        for i in range(len(df)):
                            if self.df.loc[i,'Activation'] == -1 and self.df.loc[i,'Expiry'] == self.expiryvar:
                                datevar = self.df.loc[i,'Expiry']
                                datevar_str = datevar.strftime('%Y-%m-%d') if isinstance(datevar, pd.Timestamp) else str(datevar)
                                year,day,month = datevar_str.split('-')
                                formatted_date = f"{year}{month.zfill(2)}{day}"
                                self.option_strike_type = self.df.loc[i,'Option_Type']
                                self.entry_strike = self.df.loc[i,'Entry_Strike']
                                contract = Option(symbol=self.symbol,exchange=self.exchange,lastTradeDateOrContractMonth=str(formatted_date),strike=float(self.entry_strike),right=self.option_strike_type)
                                if self.df.loc[i, 'Strike_Type'] == 'SELL':
                                    current_action = 'BUY'
                                else:
                                    current_action = 'SELL'
                                order = MarketOrder(action=current_action, totalQuantity=self.df.loc[i, 'Qty'])
                                # order.account = 'DU9727656'
                                order.account = credentials.account_no
                                order.transmit = True
                                result = self.client.placeOrder(contract, order)
                                self.df.loc[i, 'Activation'] = 0

                            elif self.df.loc[i,'Activation'] == 3 and self.df.loc[i,'Expiry'] == self.expiryvar:
                                datevar = self.df.loc[i,'Expiry']
                                datevar_str = datevar.strftime('%Y-%m-%d') if isinstance(datevar, pd.Timestamp) else str(datevar)
                                year,day,month = datevar_str.split('-')
                                formatted_date = f"{year}{month.zfill(2)}{day}"
                                self.option_strike_type = self.df.loc[i,'Option_Type']
                                self.entry_strike = self.df.loc[i,'Entry_Strike']
                                contract = Option(symbol=self.symbol,exchange=self.exchange,lastTradeDateOrContractMonth=str(formatted_date),strike=float(self.entry_strike),right=self.option_strike_type)
                                if self.df.loc[i, 'Strike_Type'] == 'SELL':
                                    current_action = 'BUY'
                                else:
                                    current_action = 'SELL'
                                order = MarketOrder(action=current_action, totalQuantity=self.df.loc[i, 'Qty'])
                                # order.account = 'DU9727656'
                                order.account = credentials.account_no
                                order.transmit = True
                                result = self.client.placeOrder(contract, order)
                                self.df.loc[i, 'Activation'] = 0
                else:
                    print("Positions are empty")

            with pd.ExcelWriter(self.path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
                self.df.to_excel(writer, sheet_name=credentials.sheet_name, index=False)
        else:
            print("No condition for closing till yet")
    

    async def get_current_market_price_futures(self, contract): 
        """
        Fetches the current market price of the given futures contract.
        """
        self.client.reqMarketDataType(3)
        ticker = self.client.reqMktData(contract, '', False, False)
        self.client.sleep(1)  
        if ticker.last is not None: 
            return ticker.last
        if ticker.close is not None: 
            return ticker.close
        
        print(ticker.last)
        return None
        # return 0.0
    
    async def show_details(self):
        result = self.ib.reqOpenOrders()
        return result
    
    async def get_bid_and_ask(self,contractmonth):
        self.client.reqMarketDataType(3)
        contract = Future(symbol=self.symbol,exchange=self.exchange,lastTradeDateOrContractMonth=contractmonth)
        test = self.ib.reqTickers(contract)
        for _,r in enumerate(test):
            bid,ask = r.bid,r.ask
            print(bid)
            print(ask)
        
        return bid,ask
    
    async def get_bid_and_ask_options(self,contractmonth,entry_strike,option_strike_type):
        self.client.reqMarketDataType(3)
        contract = Option(symbol=self.symbol,exchange=self.exchange,lastTradeDateOrContractMonth=contractmonth,strike=float(entry_strike),right=option_strike_type)
        test = self.ib.reqTickers(contract)
        for _,r in enumerate(test):
            bid,ask = r.bid,r.ask
            print(bid)
            print(ask)
        
        return bid,ask
    
    async def check_for_tp_sl(self, current_price, target_price,stop_loss,action): 
        if action == 'BUY':
            if current_price >= target_price:  # Target Price hit
                return "SELL"
            elif current_price <= stop_loss:  # Stop Loss hit
                return "SELL"

        elif action == 'SELL':
            if current_price <= target_price:  # corrected
                return "BUY"
            elif current_price >= stop_loss:  # corrected
                return "BUY"
        return None

    async def monitor_tp_sl(self): 
        self.df = pd.read_excel(self.path, sheet_name=credentials.sheet_name)  
        for i in range(len(self.df)):
            if self.df.loc[i,'Activation'] == -1 and self.df.loc[i,'Strike_Type'] == 'BUY' and self.df.loc[i,'Activation_Type'] == 1:
                datevar = self.df.loc[i, 'Expiry']
                datevar_str = datevar.strftime('%Y-%m-%d') if isinstance(datevar, pd.Timestamp) else str(datevar)
                year,day,month = datevar_str.split('-') 
                formatted_date = f"{year}{month.zfill(2)}{day}"

                self.entry_strike = self.df.loc[i,'Entry_Strike']
                self.option_strike_type = self.df.loc[i,'Option_Type']
                contract      = Future(symbol='N225M',exchange='OSE.JPN',lastTradeDateOrContractMonth=str(formatted_date))
                current_price = await self.get_current_market_price_futures(contract)

                if current_price:
                    action = await self.check_for_tp_sl(current_price, self.df['Target'].iloc[i],self.df['Stop_Loss'].iloc[i],self.df.loc[i,'Strike_Type'])
                    if action is not None:  
                        print(f"An action of sell has been triggered in row {i}")
                        contract = Option(symbol=self.symbol,exchange=self.exchange,lastTradeDateOrContractMonth=str(formatted_date),strike=float(self.entry_strike),right=self.option_strike_type)
                        order = MarketOrder(action='SELL', totalQuantity=str(self.df['Qty'].iloc[i]))
                        # order.account = 'DU9727656'
                        order.account = credentials.account_no
                        order.transmit = True
                        result = self.client.placeOrder(contract, order)
                        print(result)
                        self.df.loc[i, 'Activation'] = 0
                        print("One position is being closed")  
                    else:
                        print("No profit/loss is triggered")

            elif self.df.loc[i,'Activation'] == -1 and self.df.loc[i,'Strike_Type'] == 'SELL' and self.df.loc[i,'Activation_Type'] == 1:
                datevar = self.df.loc[i, 'Expiry']
                print(datevar)
                datevar_str = datevar.strftime('%Y-%m-%d') if isinstance(datevar, pd.Timestamp) else str(datevar)
                year,day,month = datevar_str.split('-')
                formatted_date = f"{year}{month.zfill(2)}{day}"

                self.entry_strike = self.df.loc[i,'Entry_Strike']
                self.option_strike_type = self.df.loc[i,'Option_Type']
                contract      = Future(symbol='N225M',exchange='OSE.JPN',lastTradeDateOrContractMonth=str(formatted_date))
                current_price = await self.get_current_market_price_futures(contract)

                if current_price:
                    action = await self.check_for_tp_sl(current_price, self.df['Target'].iloc[i],self.df['Stop_Loss'].iloc[i],self.df.loc[i,'Strike_Type'])
                    if action is not None:  
                        print(f"An action of buy has been triggered in row {i}")
                        contract = Option(symbol=self.symbol,exchange=self.exchange,lastTradeDateOrContractMonth=str(formatted_date),strike=float(self.entry_strike),right=self.option_strike_type)
                        order = MarketOrder(action='BUY', totalQuantity=str(self.df['Qty'].iloc[i]))
                        # order.account = 'DU9727656'
                        order.account = credentials.account_no
                        order.transmit = True
                        result = self.client.placeOrder(contract, order)
                        print(result)
                        self.df.loc[i, 'Activation'] = 0  
                    else:
                        print("No profit/loss is triggered")

        with pd.ExcelWriter(self.path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
            self.df.to_excel(writer, sheet_name=credentials.sheet_name, index=False)
    
    async def new_auto_square_off(self): # put this in async
        self.df = pd.read_excel(self.path, sheet_name=credentials.sheet_name)
        df = self.df
        current_time = datetime.now().strftime("%H:%M")
        positions = self.client.positions()
        # print(f"current time is {current_time} and closing_time is {credentials.current_time}")
        print(f"current time is {current_time} and closing_time is {credentials.closing_time}")
        # if current_time >= str(credentials.current_time):
        if current_time >= str(credentials.closing_time):
            if positions:
                for i in range(len(df)):
                    if self.df.loc[i,'Activation'] == -1 and self.df.loc[i,'Activation_Type'] == 1:
                        datevar = self.df.loc[i, 'Expiry']
                        datevar_str = datevar.strftime('%Y-%m-%d') if isinstance(datevar, pd.Timestamp) else str(datevar)
                        year,day,month = datevar_str.split('-')
                        formatted_date = f"{year}{month.zfill(2)}{day}"
                        self.entry_strike = str(self.df.loc[i,'Entry_Strike'])
                        self.option_strike_type = str(self.df.loc[i,'Option_Type'])
                        contract = Option(symbol=self.symbol,exchange=self.exchange,lastTradeDateOrContractMonth=str(formatted_date),strike=float(self.entry_strike),right=self.option_strike_type)
                        if self.df.loc[i, 'Strike_Type'] == 'SELL':
                            current_action = 'BUY'
                        else:
                            current_action = 'SELL'
                        order = MarketOrder(action=current_action, totalQuantity=self.df.loc[i, 'Qty'])
                        
                        # order.account = 'DU9727656'
                        order.account = credentials.account_no
                        order.transmit = True
                        result = self.client.placeOrder(contract, order)
                        self.df.loc[i, 'Activation'] = 0
            else:
                print("Positions are empty")
        else:
            print("The time is not for closing the market is not yet")

        with pd.ExcelWriter(self.path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
            self.df.to_excel(writer, sheet_name=credentials.sheet_name, index=False)

    async def run(self):
        print("The process has started")
        await self.connection_show()
        while True:
            await asyncio.gather(self.check_for_new_positions(),self.new_auto_square_off(),self.monitor_tp_sl(),self.close_empty_trigger_fn_lower(),self.close_empty_trigger_fn_upper())
            await asyncio.sleep(7)  # total sleep time don't change it .

if __name__ == "__main__":
    session = IBRKExcel()
    asyncio.run(session.run())


# """ports host and account"""
# port = 7497
# host = "127.0.0.1"
# # account_no = "DU9727656"
# account_no = "DU7166729"

# """symbol and paths"""
# symbol = "N225M"
# exchange = "OSE.JPN"
# # xlsx_path = r"C:\Users\vaibh\OneDrive\Desktop\New folder\Folder Python\Folder Algotrading\ibkr_excel\storage\new_order_sheet.xlsx"
# """sheet path and sheet"""
# xlsx_path_1 = r"C:\Users\vaibh\OneDrive\Desktop\New folder\Folder Python\Folder Algotrading\ibkr_excel\storage\options_order_sheet.xlsx"
# sheet_name = "Sheet1"

# """parameters for trading"""
# pause_time = 5 # set this as default - time before 
# attempts = 3 # attempts 
# closing_time = r"17:00"
# """

# attempts - the number of times the bot would place the orders ex - 3
# current_time - the time at which the trading bot would stop ex - "22:00" # closes at 10PM
# xlsx_path - path of the xlsx file (would also need to specify the sheet like Sheet1,Sheet6)
# port,host - used in login
# """

# """
# when you set the activation to 1 then normal orders for buy/selling for the futures are made once you save them then order gets placed and the activation becomes -1
# when you set the activation to 3 then goodtilldate type orders for buy/selling for the futures are made once you save them then order gets placed and the activation becomes -3
# when you don't set the take profit and stop loss and fill them with '-' and '-' respectively then based on 'buy' we will set the upper_trigger to the entry price of that column / if 'sell' then we will set the lower_trigger to the entry price of that column
# in case of multiple columns have -,- then the first one is considered this all only works when the activation is manually set to 2 otherwise if the activation is set to 0 then the column is ignored
# """


